In [1]:
# Cellule 1 - Imports
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.genmod.families import Poisson
from statsmodels.genmod.families.family import Tweedie
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def load_data():
    print("=== Chargement des données ===")
    
    # 1. Chargement des données d'entraînement et de test
    print("\nChargement des données principales:")
    train_features = pd.read_csv('data/train_input_Z61KlZo.csv', low_memory=False)
    train_target = pd.read_csv('data/train_output_DzPxaPY.csv')
    test_df = pd.read_csv('data/test_input_5qJzHrr.csv', low_memory=False)
    
    # 2. Chargement du format de soumission
    print("\nChargement du format de soumission:")
    submission_format = pd.read_csv('data/submission_csv_file_random_example_3fbDtrr (1).csv')
    
    # Vérification des colonnes du format de soumission
    expected_cols = ['ID', 'FREQ', 'CM', 'ANNEE_ASSURANCE', 'CHARGE']
    if not all(col in submission_format.columns for col in expected_cols):
        raise ValueError(f"Format de soumission incorrect. Colonnes attendues: {expected_cols}")
    
    # 3. Gestion de la colonne ANNEE_ASSURANCE
    if 'ANNEE_ASSURANCE' in train_features.columns:
        train_features = train_features.drop('ANNEE_ASSURANCE', axis=1)
    
    # 4. Fusion des données train
    train_df = pd.merge(train_features, train_target, on='ID', how='inner')
    
    # 5. Vérifications et statistiques
    print("\n=== Dimensions des données ===")
    print(f"Train features initiales: {train_features.shape}")
    print(f"Train target: {train_target.shape}")
    print(f"Train après fusion: {train_df.shape}")
    print(f"Test: {test_df.shape}")
    print(f"Format soumission: {submission_format.shape}")
    
    # 6. Vérification de la cohérence des IDs
    test_ids = set(test_df['ID'])
    submission_ids = set(submission_format['ID'])
    
    if test_ids != submission_ids:
        print("\nATTENTION: Différence entre les IDs de test et de soumission!")
        print(f"IDs uniquement dans test: {len(test_ids - submission_ids)}")
        print(f"IDs uniquement dans soumission: {len(submission_ids - test_ids)}")
    
    # 7. Vérification des colonnes
    train_cols = set(train_df.columns) - {'FREQ', 'CM', 'CHARGE', 'ID'}
    test_cols = set(test_df.columns) - {'ID'}
    
    missing_cols = test_cols - train_cols
    extra_cols = train_cols - test_cols
    
    if missing_cols:
        print(f"\nColonnes manquantes dans train: {missing_cols}")
    if extra_cols:
        print(f"\nColonnes supplémentaires dans train: {extra_cols}")
    
    # 8. Vérification des types de données
    print("\nTypes de données dans train:")
    print(train_df.dtypes.value_counts())
    
    # 9. Création d'un template de soumission vide
    submission_template = submission_format.copy()
    submission_template[['FREQ', 'CM', 'CHARGE']] = 0
    
    return {
        'train_df': train_df,
        'test_df': test_df,
        'submission_format': submission_format,
        'submission_template': submission_template,
        'stats': {
            'train_shape': train_df.shape,
            'test_shape': test_df.shape,
            'submission_shape': submission_format.shape,
            'n_features': len(train_cols),
            'missing_cols': list(missing_cols),
            'extra_cols': list(extra_cols)
        }
    }

# Exécution du chargement
data_results = load_data()

# Récupération des DataFrames
train_df = data_results['train_df']
test_df = data_results['test_df']
submission_template = data_results['submission_template']

# Affichage des statistiques détaillées
print("\n=== Résumé du chargement ===")
stats = data_results['stats']
print(f"Nombre d'observations train: {stats['train_shape'][0]}")
print(f"Nombre d'observations test: {stats['test_shape'][0]}")
print(f"Nombre de features: {stats['n_features']}")

if stats['missing_cols']:
    print("\nAttention: colonnes manquantes dans train!")
    print(stats['missing_cols'])

if stats['extra_cols']:
    print("\nColonnes supplémentaires dans train:")
    print(stats['extra_cols'])

=== Chargement des données ===

Chargement des données principales:

Chargement du format de soumission:

=== Dimensions des données ===
Train features initiales: (383610, 373)
Train target: (383610, 5)
Train après fusion: (383610, 377)
Test: (95852, 374)
Format soumission: (95852, 5)

Types de données dans train:
object     280
int64       58
float64     39
dtype: int64

=== Résumé du chargement ===
Nombre d'observations train: 383610
Nombre d'observations test: 95852
Nombre de features: 373


In [ ]:
def analyze_data_structure():
    print("=== Analyse de la structure des données ===")
    
    # Afficher les colonnes de chaque DataFrame
    print("\nColonnes du format de soumission:")
    print(data_results['submission_format'].columns.tolist())
    
    # Trouver les colonnes différentes
    train_cols = set(train_df.columns)
    test_cols = set(test_df.columns)
    
    print("\nColonnes présentes dans train mais pas dans test:")
    diff_cols = train_cols - test_cols
    print(sorted(diff_cols))
    
    # Vérifier si ANNEE_ASSURANCE est bien géré
    print("\nPrésence de ANNEE_ASSURANCE:")
    print(f"Dans train: {'ANNEE_ASSURANCE' in train_df.columns}")
    print(f"Dans test: {'ANNEE_ASSURANCE' in test_df.columns}")
    print(f"Dans soumission: {'ANNEE_ASSURANCE' in data_results['submission_format'].columns}")

# Exécuter l'analyse
analyze_data_structure()

In [ ]:
def verify_charge_calculation(train_df, submission_format):
    print("=== Vérification du calcul de CHARGE ===")
    
    # 1. Vérification dans le train
    print("\nVérification dans le train:")
    train_df['CHARGE_calc'] = train_df['FREQ'] * train_df['CM'] * train_df['ANNEE_ASSURANCE']
    train_df['diff_pct'] = abs(train_df['CHARGE_calc'] - train_df['CHARGE']) / train_df['CHARGE'] * 100
    
    print("\nStatistiques des différences dans train (en %):")
    print(train_df['diff_pct'].describe())
    
    train_problematic = train_df[train_df['diff_pct'] > 1]
    if len(train_problematic) > 0:
        print(f"\nNombre de cas problématiques dans train: {len(train_problematic)}")
        print("\nExemples de cas problématiques dans train:")
        print(train_problematic[['FREQ', 'CM', 'ANNEE_ASSURANCE', 'CHARGE', 'CHARGE_calc', 'diff_pct']].head())
    
    # 2. Vérification dans le format de soumission
    print("\nVérification dans le format de soumission:")
    submission_format['CHARGE_calc'] = (submission_format['FREQ'] * 
                                      submission_format['CM'] * 
                                      submission_format['ANNEE_ASSURANCE'])
    submission_format['diff_pct'] = (abs(submission_format['CHARGE_calc'] - 
                                       submission_format['CHARGE']) / 
                                    submission_format['CHARGE'] * 100)
    
    print("\nStatistiques des différences dans soumission (en %):")
    print(submission_format['diff_pct'].describe())
    
    sub_problematic = submission_format[submission_format['diff_pct'] > 1]
    if len(sub_problematic) > 0:
        print(f"\nNombre de cas problématiques dans soumission: {len(sub_problematic)}")
        print("\nExemples de cas problématiques dans soumission:")
        print(sub_problematic[['FREQ', 'CM', 'ANNEE_ASSURANCE', 'CHARGE', 'CHARGE_calc', 'diff_pct']].head())
    
    # 3. Statistiques globales
    print("\n=== Résumé des vérifications ===")
    stats = {
        'train': {
            'n_total': len(train_df),
            'n_problematic': len(train_problematic),
            'pct_problematic': len(train_problematic) / len(train_df) * 100,
            'max_diff': train_df['diff_pct'].max()
        },
        'submission': {
            'n_total': len(submission_format),
            'n_problematic': len(sub_problematic),
            'pct_problematic': len(sub_problematic) / len(submission_format) * 100,
            'max_diff': submission_format['diff_pct'].max()
        }
    }
    
    print("\nTrain:")
    print(f"Total: {stats['train']['n_total']}")
    print(f"Problématiques: {stats['train']['n_problematic']} ({stats['train']['pct_problematic']:.2f}%)")
    print(f"Différence max: {stats['train']['max_diff']:.2f}%")
    
    print("\nSoumission:")
    print(f"Total: {stats['submission']['n_total']}")
    print(f"Problématiques: {stats['submission']['n_problematic']} ({stats['submission']['pct_problematic']:.2f}%)")
    print(f"Différence max: {stats['submission']['max_diff']:.2f}%")
    
    return {
        'train_df': train_df,
        'submission_format': submission_format,
        'stats': stats
    }

# Exécuter la vérification
verification_results = verify_charge_calculation(train_df, data_results['submission_format'])

# Nettoyer les colonnes temporaires si nécessaire
train_df = train_df.drop(['CHARGE_calc', 'diff_pct'], axis=1, errors='ignore')
data_results['submission_format'] = data_results['submission_format'].drop(['CHARGE_calc', 'diff_pct'], axis=1, errors='ignore')

In [ ]:
def analyze_data_quality():
    print("=== Analyse approfondie de la qualité des données ===\n")
    
    # 1. Analyse des valeurs manquantes
    missing_pct = (train_df.isnull().sum() / len(train_df)) * 100
    missing_summary = pd.DataFrame({
        'Missing Count': train_df.isnull().sum(),
        'Missing Percentage': missing_pct
    }).sort_values('Missing Percentage', ascending=False)
    
    print("Colonnes avec valeurs manquantes (>50%):")
    print(missing_summary[missing_summary['Missing Percentage'] > 50])
    
    # 2. Analyse des doublons
    print(f"\nNombre de lignes dupliquées: {train_df.duplicated().sum()}")
    
    # 3. Analyse des colonnes constantes ou quasi-constantes
    constant_cols = []
    quasi_constant_cols = []
    for col in train_df.columns:
        unique_vals = train_df[col].nunique()
        if unique_vals == 1:
            constant_cols.append(col)
        elif unique_vals == 2 and train_df[col].value_counts(normalize=True).iloc[0] > 0.99:
            quasi_constant_cols.append(col)
    
    print("\nColonnes constantes:")
    print(constant_cols)
    print("\nColonnes quasi-constantes (>99% même valeur):")
    print(quasi_constant_cols)
    
    # 4. Analyse des corrélations fortes
    numeric_cols = train_df.select_dtypes(include=['int64', 'float64']).columns
    corr_matrix = train_df[numeric_cols].corr()
    high_corr_pairs = []
    
    for i in range(len(numeric_cols)):
        for j in range(i+1, len(numeric_cols)):
            if abs(corr_matrix.iloc[i,j]) > 0.95:  # Seuil de corrélation à 0.95
                high_corr_pairs.append((numeric_cols[i], numeric_cols[j], corr_matrix.iloc[i,j]))
    
    print("\nPaires de variables fortement corrélées (>0.95):")
    for pair in high_corr_pairs:
        print(f"{pair[0]} - {pair[1]}: {pair[2]:.3f}")
    
    # 5. Recommandations pour le nettoyage
    print("\n=== Recommandations pour le nettoyage des données ===")
    
    # Colonnes à potentiellement supprimer
    cols_to_drop = []
    
    # Colonnes avec trop de valeurs manquantes
    high_missing_cols = missing_summary[missing_summary['Missing Percentage'] > 80].index.tolist()
    if high_missing_cols:
        print("\nColonnes à considérer pour suppression (>80% manquantes):")
        print(high_missing_cols)
        cols_to_drop.extend(high_missing_cols)
    
    # Colonnes constantes ou quasi-constantes
    if constant_cols or quasi_constant_cols:
        print("\nColonnes à considérer pour suppression (constantes ou quasi-constantes):")
        print(constant_cols + quasi_constant_cols)
        cols_to_drop.extend(constant_cols + quasi_constant_cols)
    
    return cols_to_drop, high_corr_pairs, missing_summary

# Exécution de l'analyse
cols_to_drop, high_corr_pairs, missing_summary = analyze_data_quality()

# Visualisation des valeurs manquantes
plt.figure(figsize=(15, 6))
plt.bar(range(len(missing_summary[:20])), missing_summary['Missing Percentage'][:20])
plt.xticks(range(len(missing_summary[:20])), missing_summary.index[:20], rotation=45, ha='right')
plt.title('Top 20 des colonnes avec valeurs manquantes')
plt.ylabel('Pourcentage de valeurs manquantes')
plt.tight_layout()
plt.show()

In [ ]:
def detailed_cleaning_analysis():
    print("=== Analyse détaillée pour le nettoyage ===\n")
    
    # 1. Analyse complète des valeurs manquantes
    missing_pct = (train_df.isnull().sum() / len(train_df)) * 100
    missing_summary = pd.DataFrame({
        'Missing Count': train_df.isnull().sum(),
        'Missing Percentage': missing_pct
    }).sort_values('Missing Percentage', ascending=False)
    
    # Catégorisation des colonnes selon le pourcentage de valeurs manquantes
    extreme_missing = missing_summary[missing_summary['Missing Percentage'] > 90]
    high_missing = missing_summary[(missing_summary['Missing Percentage'] > 70) & (missing_summary['Missing Percentage'] <= 90)]
    moderate_missing = missing_summary[(missing_summary['Missing Percentage'] > 50) & (missing_summary['Missing Percentage'] <= 70)]
    
    print("1. Colonnes avec valeurs manquantes extrêmes (>90%):")
    print(extreme_missing)
    print(f"\nNombre de colonnes: {len(extreme_missing)}")
    
    print("\n2. Colonnes avec beaucoup de valeurs manquantes (70-90%):")
    print(high_missing)
    print(f"\nNombre de colonnes: {len(high_missing)}")
    
    print("\n3. Colonnes avec valeurs manquantes modérées (50-70%):")
    print(moderate_missing)
    print(f"\nNombre de colonnes: {len(moderate_missing)}")
    
    # 2. Analyse des colonnes constantes et quasi-constantes
    constant_cols = [col for col in train_df.columns if train_df[col].nunique() == 1]
    quasi_constant_cols = [col for col in train_df.columns if 
                         train_df[col].nunique() == 2 and 
                         train_df[col].value_counts(normalize=True).iloc[0] > 0.99]
    
    print("\n4. Colonnes constantes:", len(constant_cols))
    print(constant_cols)
    
    print("\n5. Colonnes quasi-constantes:", len(quasi_constant_cols))
    print(quasi_constant_cols)
    
    # 3. Recommandations
    print("\n=== Recommandations de nettoyage ===")
    
    # Colonnes à supprimer automatiquement
    auto_drop = constant_cols + list(extreme_missing.index)
    print(f"\n1. À supprimer automatiquement ({len(auto_drop)} colonnes):")
    print("- Colonnes constantes et >90% manquantes")
    
    # Colonnes à évaluer
    evaluate = list(high_missing.index) + quasi_constant_cols
    print(f"\n2. À évaluer ({len(evaluate)} colonnes):")
    print("- 70-90% manquantes")
    print("- Quasi-constantes")
    
    # Colonnes à potentiellement imputer
    impute = list(moderate_missing.index)
    print(f"\n3. À potentiellement imputer ({len(impute)} colonnes):")
    print("- 50-70% manquantes")
    
    return auto_drop, evaluate, impute

# Exécution de l'analyse détaillée
auto_drop, evaluate, impute = detailed_cleaning_analysis()

In [3]:
# 1. Définition de tous les groupes de variables
WEATHER_VARS = {
    'temp_max': [col for col in train_df.columns if any(x in col.upper() for x in 
                 ['NBJTX', 'TX', 'TXMAX', 'TXAB']) and 'TMMAX' not in col.upper()],
    
    'temp_min': [col for col in train_df.columns if any(x in col.upper() for x in 
                 ['NBJTN', 'TN', 'TNMIN', 'TNAB']) and 'TMMIN' not in col.upper()],
    
    'temp_moy': [col for col in train_df.columns if any(x in col.upper() for x in 
                 ['TM', 'TMM', 'TMMAX', 'TMMIN'])],
    
    'temp_amplitude': [col for col in train_df.columns if 'TAMPLI' in col.upper()],
    
    'wind': [col for col in train_df.columns if any(x in col.upper() for x in 
             ['FF', 'FX'])],
    
    'rain': [col for col in train_df.columns if any(x in col.upper() for x in 
             ['RR', 'RRAB'])]
}

# Redéfinition des variables bâtiment et activité
BUILDING_VARS = {
    # Surfaces
    'surface': [f'SURFACE{i}' for i in range(1, 22)],
    
    # Nombre de bâtiments (attention au NBBAT12 manquant)
    'buildings': [f'NBBAT{i}' for i in range(1, 15) if i != 12],
    
    # Caractéristiques du bâtiment
    'characteristics': [f'CARACT{i}' for i in range(1, 6)],
    
    # Types de bâtiment
    'building_type': ['TYPBAT1', 'TYPBAT2', 'ADOSS'],
    
    # Hauteurs et données BDTOPO
    'height': [
        'HAUTEUR', 'HAUTEUR_MAX',
        'BDTOPO_BAT_MAX_HAUTEUR', 'BDTOPO_BAT_MAX_HAUTEUR_MAX'
    ]
}

ACTIVITY_VARS = {
    # Multi-équipement
    'equipment': [f'EQUIPEMENT{i}' for i in range(1, 8)],
    
    # Activité et vocation
    'activity': [
        'ACTIVIT2',     # Activité
        'VOCATION',     # Vocation de l'entité professionnelle
        'TYPERS'        # Type de personne
    ],
    
    # Catégorie
    'category': ['COEFASS'],  # Retrait des variables TAILLE qui sont dans INSURANCE_VARS
    
}

INSURANCE_VARS = {
    # Indicateurs de risque
    'risk_indicators': [f'RISK{i}' for i in range(1, 14)],
    
    # Données de capitaux
    'capital': [f'KAPITAL{i}' for i in range(1, 44)],
    
    # Dérogations tarifaires
    'derogation': [f'DEROG{i}' for i in range(1, 17)],
    
    # Franchises et indemnisations
    'contract_terms': {
        'franchise': ['FRCH1', 'FRCH2'],
        'indemnisation': ['INDEM1', 'INDEM2']
    },
    
    # Données d'activité et taille
    'business_profile': {
        'chiffre_affaires': ['CA1', 'CA2', 'CA3'],
        'taille_risque': [f'TAILLE{i}' for i in range(1, 5)]  # Gardé ici car lié au profil business
    },
    
    # Historique et sinistralité
    'history': [
        'NBSINCONJ',     # Sinistralité conjoncturelle
        'NBSINSTRT',     # Sinistralité longue période
        'ANCIENNETE',    # Ancienneté du contrat
        'DUREE_REQANEUF' # Durée
    ],
    
    # Données temporelles
    'time': [
        'AN_EXERC',        # Année d'exercice
        'ANNEE_ASSURANCE'  # Nombre d'années assurées
    ]
}
GEOGRAPHIC_VARS = {
    # Distances par type d'environnement
    'distances': {
        'urban': [  # Zones urbaines
            'DISTANCE_111',  # Tissu urbain continu
            'DISTANCE_112',  # Tissu urbain discontinu
            'DISTANCE_121',  # Zones industrielles/commerciales
            'DISTANCE_122',  # Réseaux routier/ferroviaire
            'DISTANCE_123',  # Zones portuaires
            'DISTANCE_124',  # Aéroports
            'DISTANCE_131',  # Extraction de matériaux
            'DISTANCE_132',  # Décharges
            'DISTANCE_133',  # Chantiers
            'DISTANCE_141',  # Espaces verts urbains
            'DISTANCE_142'   # Équipements sportifs et de loisirs
        ],
        'natural': [  # Zones naturelles
            'DISTANCE_311',  # Forêts de feuillus
            'DISTANCE_312',  # Forêts de conifères
            'DISTANCE_313',  # Forêts mélangées
            'DISTANCE_321',  # Pelouses et pâturages naturels
            'DISTANCE_322',  # Landes et broussailles
            'DISTANCE_323',  # Végétation sclérophylle
            'DISTANCE_324',  # Forêt et végétation arbustive en mutation
            'DISTANCE_331',  # Plages, dunes et sable
            'DISTANCE_332',  # Roches nues
            'DISTANCE_333',  # Végétation clairsemée
            'DISTANCE_334',  # Zones incendiées
            'DISTANCE_335'   # Glaciers et neiges éternelles
        ],
        'water': [  # Zones aquatiques
            'DISTANCE_411',  # Marais intérieurs
            'DISTANCE_412',  # Tourbières
            'DISTANCE_421',  # Marais maritimes
            'DISTANCE_422',  # Marais salants
            'DISTANCE_423',  # Zones intertidales
            'DISTANCE_511',  # Cours d'eau
            'DISTANCE_512',  # Plans d'eau
            'DISTANCE_521',  # Lagunes littorales
            'DISTANCE_522',  # Estuaires
            'DISTANCE_523'   # Mers et océans
        ],
        'other': [  # Autres distances
            'DISTANCE_211',  # Terres arables hors périmètres d'irrigation
            'DISTANCE_212',  # Périmètres irrigués en permanence
            'DISTANCE_213',  # Rizières
            'DISTANCE_221',  # Vignobles
            'DISTANCE_222',  # Vergers et petits fruits
            'DISTANCE_223',  # Oliveraies
            'DISTANCE_231',  # Prairies et autres surfaces toujours en herbe
            'DISTANCE_242',  # Systèmes culturaux et parcellaires complexes
            'DISTANCE_243',  # Surfaces agricoles avec végétation naturelle
            'DISTANCE_244'   # Territoires agroforestiers
        ],
        'general': [  # Distances générales
            'DISTANCE_1',    # Distance générale 1
            'DISTANCE_2'     # Distance générale 2
        ]
    },
    
    # Proportions par type d'environnement
    'proportions': {
        'urban': [  # Zones urbaines
            'PROPORTION_11',  # Zones urbanisées
            'PROPORTION_12',  # Zones industrielles et commerciales
            'PROPORTION_13',  # Mines, décharges et chantiers
            'PROPORTION_14'   # Espaces verts artificialisés
        ],
        'agricultural': [  # Zones agricoles
            'PROPORTION_21',  # Terres arables
            'PROPORTION_22',  # Cultures permanentes
            'PROPORTION_23',  # Prairies
            'PROPORTION_24'   # Zones agricoles hétérogènes
        ],
        'natural': [  # Zones naturelles
            'PROPORTION_31',  # Forêts
            'PROPORTION_32',  # Milieux à végétation arbustive et/ou herbacée
            'PROPORTION_33'   # Espaces ouverts, sans ou avec peu de végétation
        ],
        'water': [  # Zones aquatiques
            'PROPORTION_41',  # Zones humides intérieures
            'PROPORTION_42',  # Zones humides maritimes
            'PROPORTION_51',  # Eaux continentales
            'PROPORTION_52'   # Eaux maritimes
        ]
    },
    
    # Données d'altitude et zones
    'topography': {
        'altitude': [f'ALTITUDE_{i}' for i in range(1, 6)],  # Différents niveaux d'altitude
        'zone_info': [
            'ZONE',          # Zone géographique
            'ZONE_VENT',     # Zone de vent
            'ESPINSEE'       # Espace INSEE
        ]
    },
    
    # Données d'urgence
    'emergency': ['NB_CASERNES']  # Nombre de casernes de pompiers à proximité
}
DEMOGRAPHIC_VARS = {
    # Variables ménages
    'household': {
        'general': ['MEN'],  # Nombre total de ménages
        'composition': [
            'MEN_1IND',      # Ménages d'un seul individu
            'MEN_5IND',      # Ménages de 5 individus ou plus
            'MEN_FMP'        # Ménages monoparentaux
        ],
        'housing_type': [
            'MEN_COLL',      # Ménages en logements collectifs
            'MEN_MAIS',      # Ménages en maison
            'MEN_PROP'       # Ménages propriétaires
        ],
        'economic': [
            'MEN_PAUV',      # Ménages pauvres
            'MEN_SURF'       # Surface des logements du carreau
        ]
    },
    
    # Variables logement
    'housing': {
        'construction_period': [
            'LOG_AVA1',      # Avant année A1
            'LOG_A1_A2',     # Entre A1 et A2
            'LOG_A2_A3',     # Entre A2 et A3
            'LOG_APA3'       # Après A3
        ],
        'special_categories': [
            'LOG_INC',       # Date de construction inconnue
            'LOG_SOC'        # Logements sociaux
        ]
    },
    
    # Variables individus
    'individual': {
        'total': ['IND'],    # Nombre total d'individus
        'age_groups': [
            'IND_0_Y1',      # 0 à Y1 ans
            'IND_Y1_Y2',     # Y1 à Y2 ans
            'IND_Y2_Y3',     # Y2 à Y3 ans
            'IND_Y3_Y4',     # Y3 à Y4 ans
            'IND_Y4_Y5',     # Y4 à Y5 ans
            'IND_Y5_Y6',     # Y5 à Y6 ans
            'IND_Y6_Y7',     # Y6 à Y7 ans
            'IND_Y7_Y8',     # Y7 à Y8 ans
            'IND_Y8_Y9',     # Y8 à Y9 ans
            'IND_Y9'         # Y9 ans ou plus
        ],
        'other': [
            'IND_INC',       # Âge inconnu
            'IND_SNV'        # Niveaux de vie winsorisés
        ]
    }
}
TARGET_VARS = {
    'primary': {
        'frequency': ['FREQ'],      # Fréquence
        'cost': ['CM'],            # Coût moyen
        'total': ['CHARGE'],       # Charge totale
        'period': ['ANNEE_ASSURANCE']  # Période d'assurance
    },
    'definitions': {
        'FREQ': 'NOMBRE DE SINISTRES / ANNEE ASSURANCE',
        'CM': 'CHARGE SINISTRES / NOMBRE DE SINISTRES',
        'CHARGE': 'FREQ * CM * ANNEE_ASSURANCE',
        'ANNEE_ASSURANCE': 'Durée de la période d\'assurance en années'
    }
}
ID_VARS = {
    'identifiers': ['ID'],
    'properties': {
        'unique': True,          # L'ID doit être unique
        'not_null': True,        # Pas de valeurs nulles autorisées
        'type': 'str'           # Type attendu
    }
}



In [ ]:
def verify_weather_vars():
    print("=== Vérification des variables météo ===\n")
    
    # 1. Vérifier les doublons globaux
    all_weather_vars = []
    for category, cols in WEATHER_VARS.items():
        all_weather_vars.extend(cols)
    
    duplicates = [var for var in set(all_weather_vars) 
                 if all_weather_vars.count(var) > 1]
    
    if duplicates:
        print("ATTENTION! Variables en double:")
        for var in duplicates:
            categories = [cat for cat, cols in WEATHER_VARS.items() if var in cols]
            print(f"- {var} présent dans: {categories}")
    
    # 2. Vérifier chaque catégorie
    for category, cols in WEATHER_VARS.items():
        print(f"\n{category.upper()}:")
        print(f"Nombre de variables: {len(cols)}")
        
        # Vérifier les préfixes
        prefixes = set([col.split('_')[0] for col in cols])
        print("Préfixes trouvés:", prefixes)
        
        # Exemples de variables
        print("Exemples de variables:", sorted(cols)[:3])
        
        # Vérifier l'existence dans le DataFrame
        missing_in_df = [col for col in cols if col not in train_df.columns]
        if missing_in_df:
            print(f"ATTENTION! Variables non trouvées dans le DataFrame:")
            print(missing_in_df)
        
        # Vérifier les valeurs manquantes
        existing_cols = [col for col in cols if col in train_df.columns]
        if existing_cols:
            missing_pct = (train_df[existing_cols].isnull().mean() * 100).mean()
            print(f"Pourcentage moyen de valeurs manquantes: {missing_pct:.2f}%")
    
    # 3. Vérifier si des variables météo n'ont pas été classées
    weather_patterns = {
        'temp': ['TX', 'TN', 'TM', 'TEMP'],
        'wind': ['FF', 'FX'],
        'rain': ['RR', 'RRAB'],
        'amplitude': ['TAMPLI']
    }
    
    unclassified = []
    for pattern_type, patterns in weather_patterns.items():
        for col in train_df.columns:
            if any(pattern in col.upper() for pattern in patterns):
                if col not in all_weather_vars:
                    unclassified.append((col, pattern_type))
    
    if unclassified:
        print("\nVariables météo potentiellement non classées:")
        for col, type_ in unclassified:
            print(f"- {col} (type: {type_})")
    
    # 4. Résumé
    print("\n=== Résumé des variables météo ===")
    print(f"Total variables classées: {len(set(all_weather_vars))}")
    print(f"Variables en double: {len(duplicates)}")
    print(f"Variables non classées: {len(unclassified)}")
    
    return {
        'classified_vars': set(all_weather_vars),
        'duplicates': duplicates,
        'unclassified': unclassified,
        'missing_in_df': {cat: [col for col in cols if col not in train_df.columns] 
                         for cat, cols in WEATHER_VARS.items()}
    }

# Exécuter la vérification
weather_results = verify_weather_vars()

In [ ]:
def verify_building_and_activity_vars():
    print("=== Vérification des variables bâtiment et activité ===\n")
    
    # Collecter toutes les variables pour vérification des doublons
    all_building_vars = []
    all_activity_vars = []
    
    # Vérifier les deux groupes
    for group_name, group_vars, all_vars_list in [
        ("BUILDING", BUILDING_VARS, all_building_vars), 
        ("ACTIVITY", ACTIVITY_VARS, all_activity_vars)
    ]:
        print(f"\n{group_name}:")
        
        for category, vars_list in group_vars.items():
            print(f"\n  {category}:")
            print(f"  Nombre de variables définies: {len(vars_list)}")
            print("  Variables:", vars_list)
            
            # Vérifier l'existence
            existing_vars = [var for var in vars_list if var in train_df.columns]
            missing_vars = [var for var in vars_list if var not in train_df.columns]
            
            print(f"  Variables trouvées: {len(existing_vars)}/{len(vars_list)}")
            
            if missing_vars:
                print(f"  ATTENTION! Variables manquantes: {missing_vars}")
            
            # Vérifier les valeurs manquantes pour les variables existantes
            if existing_vars:
                missing_pct = (train_df[existing_vars].isnull().mean() * 100).mean()
                print(f"  Pourcentage moyen de valeurs manquantes: {missing_pct:.2f}%")
            
            all_vars_list.extend(vars_list)
    
    # Vérifier les doublons entre les deux groupes
    building_vars = set(all_building_vars)
    activity_vars = set(all_activity_vars)
    duplicates = building_vars.intersection(activity_vars)
    
    if duplicates:
        print("\nATTENTION! Variables en double entre BUILDING et ACTIVITY:")
        print(duplicates)
    
    # 3. Vérifier les variables non classées par type
    patterns = {
        'surface': 'SURFACE',
        'batiment': 'NBBAT',
        'caracteristique': 'CARACT',
        'type_batiment': 'TYPBAT',
        'hauteur': 'HAUTEUR',
        'bdtopo': 'BDTOPO',
        'equipement': 'EQUIPEMENT',
        'activite': 'ACTIVIT',
        'vocation': 'VOCATION',
        'type_personne': 'TYPERS'
    }
    
    unclassified = {}
    all_defined_vars = building_vars.union(activity_vars)
    
    for pattern_name, pattern in patterns.items():
        pattern_vars = [col for col in train_df.columns if pattern in col 
                       and col not in all_defined_vars]
        if pattern_vars:
            unclassified[pattern_name] = pattern_vars
    
    if unclassified:
        print("\nVariables non classées par type:")
        for pattern_name, vars_list in unclassified.items():
            print(f"\n{pattern_name.upper()}:")
            print(f"Nombre: {len(vars_list)}")
            print("Variables:", vars_list)
    
    # Résumé final
    total_unclassified = sum(len(vars_) for vars_ in unclassified.values())
    
    print("\n=== Résumé ===")
    print(f"Variables bâtiment: {len(building_vars)}")
    print(f"Variables activité: {len(activity_vars)}")
    print(f"Variables en double: {len(duplicates)}")
    print(f"Variables non classées: {total_unclassified}")
    
    return {
        'building_vars': building_vars,
        'activity_vars': activity_vars,
        'duplicates': duplicates,
        'unclassified': unclassified,
        'stats': {
            'building': {cat: len(vars_) for cat, vars_ in BUILDING_VARS.items()},
            'activity': {cat: len(vars_) for cat, vars_ in ACTIVITY_VARS.items()},
            'total_unclassified': total_unclassified
        }
    }

# Exécuter la vérification
building_activity_results = verify_building_and_activity_vars()

In [ ]:
import re  # Pour les expressions régulières
def verify_insurance_vars():
    """
    Vérifie la cohérence et la présence des variables d'assurance.
    """
    print("=== Vérification des variables d'assurance ===\n")
    
    # 1. Collecter toutes les variables définies
    all_defined_vars = []
    
    # 2. Vérifier chaque catégorie principale
    for category, content in INSURANCE_VARS.items():
        # 2.1 Traitement des catégories avec sous-catégories
        if isinstance(content, dict):
            print(f"\n{category.upper()}:")
            for subcat, vars_list in content.items():
                print(f"\n  {subcat}:")
                verify_variable_group(vars_list, all_defined_vars)
        
        # 2.2 Traitement des catégories simples
        else:
            print(f"\n{category.upper()}:")
            verify_variable_group(content, all_defined_vars)
    
    # 3. Vérifier les doublons
    duplicates = find_duplicates(all_defined_vars)
    
    # 4. Vérifier les variables non classées
    unclassified = find_unclassified_vars(all_defined_vars)
    
    # 5. Générer le résumé
    print_summary(all_defined_vars, duplicates, unclassified)
    
    return generate_results(all_defined_vars, duplicates, unclassified)

def verify_variable_group(vars_list, all_vars):
    """
    Vérifie un groupe de variables et met à jour la liste globale.
    """
    print(f"  Nombre de variables définies: {len(vars_list)}")
    print("  Variables:", vars_list)
    
    # Vérifier l'existence
    existing_vars = [var for var in vars_list if var in train_df.columns]
    missing_vars = [var for var in vars_list if var not in train_df.columns]
    
    print(f"  Variables trouvées: {len(existing_vars)}/{len(vars_list)}")
    
    if missing_vars:
        print(f"  ATTENTION! Variables manquantes: {missing_vars}")
    
    # Vérifier les valeurs manquantes
    if existing_vars:
        missing_pct = (train_df[existing_vars].isnull().mean() * 100).mean()
        print(f"  Pourcentage moyen de valeurs manquantes: {missing_pct:.2f}%")
    
    all_vars.extend(vars_list)

def find_duplicates(all_vars):
    """
    Identifie les variables en double.
    """
    return [var for var in set(all_vars) if all_vars.count(var) > 1]

def find_unclassified_vars(all_vars):
    """
    Identifie les variables d'assurance non classées.
    """
    insurance_patterns = {
        'risk': '^RISK[0-9]+$',
        'capital': '^KAPITAL[0-9]+$',
        'derogation': '^DEROG[0-9]+$',
        'franchise': '^FRCH[0-9]+$',
        'indemnisation': '^INDEM[0-9]+$',
        'chiffre_affaires': '^CA[0-9]+$',
        'sinistre': '^NBSIN',
        'anciennete': '^ANCIEN',
        'exercice': '^(AN_)?EXERC'
    }
    
    # Exclure les patterns d'autres groupes
    other_patterns = ['VOCATION', 'CARACT', 'NB_CASERNES', 'EQUIPEMENT', 'SURFACE']
    
    unclassified = {}
    for pattern_name, pattern in insurance_patterns.items():
        pattern_vars = [col for col in train_df.columns 
                       if re.match(pattern, col) and col not in all_vars
                       and not any(other in col for other in other_patterns)]
        if pattern_vars:
            unclassified[pattern_name] = pattern_vars
    
    return {k: v for k, v in unclassified.items() if v}

def print_summary(all_vars, duplicates, unclassified):
    """
    Affiche le résumé de la vérification.
    """
    if unclassified:
        print("\nVariables non classées par type:")
        for pattern_name, vars_list in unclassified.items():
            print(f"\n{pattern_name.upper()}:")
            print(f"Nombre: {len(vars_list)}")
            print("Variables:", vars_list)
    
    total_unclassified = sum(len(vars_) for vars_ in unclassified.values())
    
    print("\n=== Résumé ===")
    print(f"Total variables définies: {len(all_vars)}")
    print(f"Variables uniques: {len(set(all_vars))}")
    print(f"Doublons: {len(duplicates)}")
    if duplicates:
        print("Variables en double:", duplicates)
    print(f"Variables non classées: {total_unclassified}")

def generate_results(all_vars, duplicates, unclassified):
    """
    Génère le dictionnaire de résultats.
    """
    total_unclassified = sum(len(vars_) for vars_ in unclassified.values())
    
    return {
        'all_vars': set(all_vars),
        'duplicates': duplicates,
        'unclassified': unclassified,
        'missing_vars': {cat: [var for var in (content if isinstance(content, list) 
                                              else sum(content.values(), [])) 
                              if var not in train_df.columns] 
                        for cat, content in INSURANCE_VARS.items()},
        'stats': {
            'total_defined': len(all_vars),
            'total_unique': len(set(all_vars)),
            'total_duplicates': len(duplicates),
            'total_unclassified': total_unclassified
        }
    }

# Exécuter la vérification
insurance_results = verify_insurance_vars()

In [ ]:
def verify_geographic_vars():
    print("=== Vérification des variables géographiques ===\n")
    
    # Aplatir la structure imbriquée
    all_vars = []
    category_vars = {}  # Pour suivre les variables par catégorie
    
    # 1. Collecter toutes les variables
    for main_category, content in GEOGRAPHIC_VARS.items():
        category_vars[main_category] = []
        if isinstance(content, dict):
            for subcategory, vars_list in content.items():
                all_vars.extend(vars_list)
                category_vars[main_category].extend(vars_list)
        else:
            all_vars.extend(content)
            category_vars[main_category].extend(content)
    
    # 2. Vérifier chaque catégorie
    for main_category, content in GEOGRAPHIC_VARS.items():
        print(f"\n{main_category.upper()}:")
        
        if isinstance(content, dict):
            for subcategory, vars_list in content.items():
                print(f"\n  {subcategory}:")
                print(f"  Nombre de variables définies: {len(vars_list)}")
                if len(vars_list) > 3:
                    print(f"  Exemples de variables: {vars_list[:3]} ...")
                else:
                    print(f"  Variables: {vars_list}")
                
                # Vérifier l'existence
                existing_vars = [var for var in vars_list if var in train_df.columns]
                missing_vars = [var for var in vars_list if var not in train_df.columns]
                
                print(f"  Variables trouvées: {len(existing_vars)}/{len(vars_list)}")
                
                if missing_vars:
                    print(f"  ATTENTION! Variables manquantes ({len(missing_vars)}):")
                    for var in missing_vars[:5]:  # Montrer les 5 premiers exemples
                        print(f"    - {var}")
                    if len(missing_vars) > 5:
                        print(f"    ... et {len(missing_vars)-5} autres")
                
                # Vérifier les valeurs manquantes
                if existing_vars:
                    missing_pct = (train_df[existing_vars].isnull().mean() * 100).mean()
                    print(f"  Pourcentage moyen de valeurs manquantes: {missing_pct:.2f}%")
        else:
            print(f"Nombre de variables: {len(content)}")
            print("Variables:", content)
    
    # 3. Vérifier les variables non classées
    patterns = {
        'distance': 'DISTANCE_',
        'proportion': 'PROPORTION_',
        'altitude': 'ALTITUDE_',
        'zone': 'ZONE'
    }
    
    unclassified = {}
    for pattern_name, pattern in patterns.items():
        pattern_vars = [col for col in train_df.columns if pattern in col 
                       and col not in all_vars]
        if pattern_vars:
            unclassified[pattern_name] = pattern_vars
    
    if unclassified:
        print("\nVariables non classées par type:")
        for pattern_name, vars_list in unclassified.items():
            print(f"\n{pattern_name.upper()}:")
            print(f"Nombre: {len(vars_list)}")
            print("Exemples:", vars_list[:5], "..." if len(vars_list) > 5 else "")
    
    # 4. Résumé final
    print("\n=== Résumé de la vérification géographique ===")
    print(f"Total variables définies: {len(all_vars)}")
    
    existing_total = len([var for var in all_vars if var in train_df.columns])
    print(f"Variables existantes: {existing_total}")
    print(f"Variables manquantes: {len(all_vars) - existing_total}")
    
    total_unclassified = sum(len(vars_) for vars_ in unclassified.values())
    print(f"Variables non classées: {total_unclassified}")
    
    # 5. Statistiques par catégorie
    print("\nStatistiques par catégorie principale:")
    for category, vars_list in category_vars.items():
        existing = len([var for var in vars_list if var in train_df.columns])
        print(f"{category}: {existing}/{len(vars_list)} variables trouvées")
    
    return {
        'all_vars': set(all_vars),
        'by_category': category_vars,
        'unclassified': unclassified,
        'stats': {
            'total_defined': len(all_vars),
            'total_existing': existing_total,
            'total_missing': len(all_vars) - existing_total,
            'total_unclassified': total_unclassified
        }
    }

# Exécuter la vérification
geo_results = verify_geographic_vars()

In [ ]:
def verify_demographic_vars():
    print("=== Vérification des variables démographiques ===\n")
    
    # Aplatir la structure imbriquée
    all_vars = []
    for main_category, subcategories in DEMOGRAPHIC_VARS.items():
        for subcat, vars_list in subcategories.items():
            all_vars.extend(vars_list)
    
    # Vérifier chaque catégorie et sous-catégorie
    for main_category, subcategories in DEMOGRAPHIC_VARS.items():
        print(f"\n{main_category.upper()}:")
        
        for subcat, vars_list in subcategories.items():
            print(f"\n  {subcat}:")
            print(f"  Nombre de variables définies: {len(vars_list)}")
            print("  Variables définies:", vars_list)
            
            # Vérifier l'existence
            existing_vars = [var for var in vars_list if var in train_df.columns]
            missing_vars = [var for var in vars_list if var not in train_df.columns]
            
            print(f"  Variables trouvées: {len(existing_vars)}/{len(vars_list)}")
            
            if missing_vars:
                print(f"  ATTENTION! Variables manquantes: {missing_vars}")
            
            # Vérifier les valeurs manquantes seulement pour les variables existantes
            if existing_vars:
                missing_pct = (train_df[existing_vars].isnull().mean() * 100).mean()
                print(f"  Pourcentage moyen de valeurs manquantes: {missing_pct:.2f}%")
    
    # Patterns pour identifier les variables démographiques
    demo_patterns = {
        'menage': '^MEN_',
        'logement': '^LOG_',
        'individu': '^IND_'
    }
    
    # Exclure les patterns d'autres groupes
    other_patterns = ['INDEM', 'EQUIPEMENT', 'ACTIVIT', 'SURFACE', 'CARACT']
    
    # Vérifier les variables non classées
    unclassified = {}
    for pattern_name, pattern in demo_patterns.items():
        pattern_vars = [col for col in train_df.columns 
                       if re.match(pattern, col) 
                       and col not in all_vars
                       and not any(other in col for other in other_patterns)]
        if pattern_vars:
            unclassified[pattern_name] = pattern_vars
    
    # Afficher les variables non classées par type
    if unclassified:
        print("\nVariables non classées par type:")
        for pattern_name, vars_list in unclassified.items():
            if vars_list:  # N'afficher que si la liste n'est pas vide
                print(f"\n{pattern_name.upper()}:")
                print(f"Nombre: {len(vars_list)}")
                print("Variables:", vars_list)
    
    # Résumé final
    total_unclassified = sum(len(vars_) for vars_ in unclassified.values())
    
    print("\n=== Résumé de la vérification démographique ===")
    existing_vars = [var for var in all_vars if var in train_df.columns]
    missing_vars = [var for var in all_vars if var not in train_df.columns]
    print(f"Variables définies: {len(all_vars)}")
    print(f"Variables existantes: {len(existing_vars)}")
    print(f"Variables manquantes: {len(missing_vars)}")
    print(f"Variables non classées: {total_unclassified}")
    
    return {
        'all_defined_vars': all_vars,
        'existing_vars': existing_vars,
        'missing_vars': missing_vars,
        'unclassified': unclassified,
        'stats': {
            'total_defined': len(all_vars),
            'total_existing': len(existing_vars),
            'total_missing': len(missing_vars),
            'total_unclassified': total_unclassified
        }
    }

# Exécuter la vérification
demo_results = verify_demographic_vars()

In [ ]:
def verify_target_vars():
    print("=== Vérification des variables cibles ===\n")
    
    # Aplatir la structure pour les variables
    all_vars = []
    for category in TARGET_VARS['primary'].values():
        all_vars.extend(category)
    
    # Vérifier la présence des variables
    for category, vars_list in TARGET_VARS['primary'].items():
        print(f"\n{category.upper()}:")
        print(f"Variables: {vars_list}")
        
        # Vérifier l'existence
        missing = [var for var in vars_list if var not in train_df.columns]
        if missing:
            print(f"ATTENTION! Variables manquantes: {missing}")
        
        # Vérifier les valeurs manquantes
        if vars_list:
            missing_pct = (train_df[vars_list].isnull().mean() * 100).mean()
            print(f"Pourcentage moyen de valeurs manquantes: {missing_pct:.2f}%")
        
        # Afficher la définition
        for var in vars_list:
            if var in TARGET_VARS['definitions']:
                print(f"Définition de {var}: {TARGET_VARS['definitions'][var]}")
    
    # Vérifier les variables non classées
    target_patterns = {
        'frequence': '^FREQ',
        'cout_moyen': '^CM',
        'charge': '^CHARGE',
        'annee': '^ANNEE_ASSURANCE'
    }
    
    # Exclure les patterns d'autres groupes
    other_patterns = ['FRCH', 'CARACT', 'SURFACE']
    
    unclassified = {}
    for pattern_name, pattern in target_patterns.items():
        pattern_vars = [col for col in train_df.columns 
                       if re.match(pattern, col) 
                       and col not in all_vars
                       and not any(other in col for other in other_patterns)]
        if pattern_vars:
            unclassified[pattern_name] = pattern_vars
    
    if unclassified:
        print("\nVariables non classées par type:")
        for pattern_name, vars_list in unclassified.items():
            if vars_list:
                print(f"\n{pattern_name.upper()}:")
                print(f"Nombre: {len(vars_list)}")
                print("Variables:", vars_list)
    
    # Vérifier la cohérence des calculs
    if all(var in train_df.columns for var in ['FREQ', 'CM', 'CHARGE', 'ANNEE_ASSURANCE']):
        # Calculer CHARGE théorique
        charge_calc = train_df['FREQ'] * train_df['CM'] * train_df['ANNEE_ASSURANCE']
        
        # Comparer avec CHARGE réelle
        diff_pct = abs(charge_calc - train_df['CHARGE']).mean() / train_df['CHARGE'].mean() * 100
        print(f"\nDifférence moyenne entre CHARGE calculée et réelle: {diff_pct:.2f}%")
    
    # Statistiques descriptives
    print("\nStatistiques descriptives:")
    for var in all_vars:
        if var in train_df.columns:
            stats = train_df[var].describe()
            print(f"\n{var}:")
            print(f"  Min: {stats['min']:.2f}")
            print(f"  Max: {stats['max']:.2f}")
            print(f"  Moyenne: {stats['mean']:.2f}")
            print(f"  Écart-type: {stats['std']:.2f}")
    
    # Résumé final
    total_unclassified = sum(len(vars_) for vars_ in unclassified.values())
    print("\n=== Résumé ===")
    print(f"Variables définies: {len(all_vars)}")
    print(f"Variables existantes: {len([var for var in all_vars if var in train_df.columns])}")
    print(f"Variables manquantes: {len([var for var in all_vars if var not in train_df.columns])}")
    print(f"Variables non classées: {total_unclassified}")
    
    return {
        'variables': all_vars,
        'definitions': TARGET_VARS['definitions'],
        'unclassified': unclassified,
        'missing_stats': {var: train_df[var].isnull().mean() * 100 
                         for var in all_vars if var in train_df.columns},
        'stats': {
            'total_defined': len(all_vars),
            'total_existing': len([var for var in all_vars if var in train_df.columns]),
            'total_missing': len([var for var in all_vars if var not in train_df.columns]),
            'total_unclassified': total_unclassified
        }
    }

# Exécuter la vérification
target_results = verify_target_vars()

In [ ]:
def verify_id_vars():
    """
    Vérifie la cohérence et la présence des variables d'identification.
    """
    print("=== Vérification des variables d'identification ===\n")
    
    id_vars = ID_VARS['identifiers']
    properties = ID_VARS['properties']
    
    # 1. Vérifier la présence des variables
    missing = [var for var in id_vars if var not in train_df.columns]
    if missing:
        print(f"ATTENTION! Variables manquantes: {missing}")
        return
    
    # 2. Vérifier chaque identifiant
    for id_var in id_vars:
        verify_single_id(id_var, properties)
    
    # 3. Vérifier les variables non classées
    unclassified = find_unclassified_vars(id_vars)
    
    # 4. Afficher les statistiques générales
    print_id_statistics(id_vars)
    
    # 5. Générer et afficher le résumé final
    results = generate_results(id_vars, unclassified)
    print_final_summary(results)
    
    return results

def verify_single_id(id_var, properties):
    """Vérifie un identifiant spécifique."""
    print(f"\nVérification de {id_var}:")
    
    # Vérifier l'unicité
    if properties['unique']:
        check_uniqueness(id_var)
    
    # Vérifier les valeurs nulles
    if properties['not_null']:
        check_null_values(id_var)
    
    # Vérifier le type de données
    if properties['type'] == 'str':
        check_data_type(id_var)
    
    # Vérifier la cohérence entre les ensembles
    check_dataset_consistency(id_var)

def check_uniqueness(id_var):
    """Vérifie l'unicité des identifiants."""
    n_unique = train_df[id_var].nunique()
    n_total = len(train_df)
    if n_unique != n_total:
        print(f"ATTENTION! {id_var} n'est pas unique:")
        print(f"  {n_unique} valeurs uniques pour {n_total} lignes")
        duplicates = train_df[train_df[id_var].duplicated(keep=False)]
        if not duplicates.empty:
            print("  Exemples de doublons:")
            print(duplicates[id_var].head())

def check_null_values(id_var):
    """Vérifie la présence de valeurs nulles."""
    n_null = train_df[id_var].isnull().sum()
    if n_null > 0:
        print(f"ATTENTION! {n_null} valeurs nulles trouvées")

def check_data_type(id_var):
    """Vérifie le type de données."""
    if not pd.api.types.is_string_dtype(train_df[id_var]):
        print(f"ATTENTION! {id_var} n'est pas de type string")
        print(f"Type actuel: {train_df[id_var].dtype}")

def check_dataset_consistency(id_var):
    """Vérifie la cohérence entre train, test et soumission."""
    train_ids = set(train_df[id_var])
    test_ids = set(test_df[id_var])
    submission_ids = set(data_results['submission_format'][id_var])
    
    # Vérifier l'intersection train/test
    common_train_test = train_ids.intersection(test_ids)
    if common_train_test:
        print(f"\nATTENTION! IDs communs entre train et test: {len(common_train_test)}")
        print("Exemples:", list(common_train_test)[:5])
    
    # Vérifier la cohérence test/soumission
    check_submission_consistency(test_ids, submission_ids)
    
    return train_ids, test_ids, submission_ids, common_train_test

def check_submission_consistency(test_ids, submission_ids):
    """Vérifie la cohérence avec le fichier de soumission."""
    missing_in_submission = test_ids - submission_ids
    extra_in_submission = submission_ids - test_ids
    if missing_in_submission or extra_in_submission:
        print("\nIncohérences entre test et soumission:")
        if missing_in_submission:
            print(f"IDs manquants dans soumission: {len(missing_in_submission)}")
        if extra_in_submission:
            print(f"IDs supplémentaires dans soumission: {len(extra_in_submission)}")
    return missing_in_submission, extra_in_submission

def find_unclassified_vars(id_vars):
    """Identifie les variables d'identification non classées."""
    id_patterns = {
        'identifiant': '^ID',
        'reference': '^REF',
        'numero': '^NUM'
    }
    
    # Exclure les patterns d'autres groupes
    other_patterns = ['INDEM', 'IND_', 'INDICE']
    
    unclassified = {}
    for pattern_name, pattern in id_patterns.items():
        pattern_vars = [col for col in train_df.columns 
                       if re.match(pattern, col) 
                       and col not in id_vars
                       and not any(other in col for other in other_patterns)]
        if pattern_vars:
            unclassified[pattern_name] = pattern_vars
    
    if unclassified:
        print("\nVariables non classées par type:")
        for pattern_name, vars_list in unclassified.items():
            if vars_list:
                print(f"\n{pattern_name.upper()}:")
                print(f"Nombre: {len(vars_list)}")
                print("Variables:", vars_list)
    
    return unclassified

def print_id_statistics(id_vars):
    """Affiche les statistiques des identifiants."""
    print("\n=== Statistiques des IDs ===")
    for id_var in id_vars:
        print(f"\n{id_var}:")
        print(f"Train: {train_df[id_var].nunique()} valeurs uniques")
        print(f"Test: {test_df[id_var].nunique()} valeurs uniques")
        print(f"Soumission: {data_results['submission_format'][id_var].nunique()} valeurs uniques")
        
        if pd.api.types.is_string_dtype(train_df[id_var]):
            train_lengths = train_df[id_var].str.len()
            print(f"Longueur min: {train_lengths.min()}")
            print(f"Longueur max: {train_lengths.max()}")
            if train_lengths.min() != train_lengths.max():
                print("ATTENTION! Longueurs variables")

def generate_results(id_vars, unclassified):
    """Génère le dictionnaire des résultats."""
    id_var = id_vars[0]  # Premier identifiant
    train_ids = set(train_df[id_var])
    test_ids = set(test_df[id_var])
    submission_ids = set(data_results['submission_format'][id_var])
    
    total_unclassified = sum(len(vars_) for vars_ in unclassified.values())
    
    return {
        'train_ids': train_ids,
        'test_ids': test_ids,
        'submission_ids': submission_ids,
        'unclassified': unclassified,
        'stats': {
            'train_unique': len(train_ids),
            'test_unique': len(test_ids),
            'submission_unique': len(submission_ids),
            'total_unclassified': total_unclassified
        }
    }

def print_final_summary(results):
    """Affiche le résumé final."""
    print("\n=== Résumé final ===")
    stats = results['stats']
    print(f"Train: {stats['train_unique']} IDs uniques")
    print(f"Test: {stats['test_unique']} IDs uniques")
    print(f"Soumission: {stats['submission_unique']} IDs uniques")
    print(f"Variables non classées: {stats['total_unclassified']}")

# Exécuter la vérification
id_results = verify_id_vars()

In [ ]:
def verify_variable_classification():
    print("=== Vérification globale de la classification des variables ===")
    
    # 1. Collecter toutes les variables de chaque groupe
    group_vars = {
        'Weather': sum(WEATHER_VARS.values(), []),
        'Building': sum(BUILDING_VARS.values(), []),
        'Insurance': sum([v if isinstance(v, list) else sum(v.values(), []) 
                         for v in INSURANCE_VARS.values()], []),
        'Geographic': sum([v if isinstance(v, list) else sum(v.values(), []) 
                          for v in GEOGRAPHIC_VARS.values()], []),
        'Demographic': sum([sum(v.values(), []) for v in DEMOGRAPHIC_VARS.values()], []),
        'Activity': sum(ACTIVITY_VARS.values(), []),
        'Target': sum(TARGET_VARS['primary'].values(), []),
        'ID': ID_VARS['identifiers']
    }
    
    # 2. Vérifier les répétitions entre groupes
    all_vars = set(train_df.columns)
    classified_vars = set()
    duplicates = {}
    
    for group, vars_list in group_vars.items():
        # Vérifier les répétitions internes au groupe
        internal_dupes = [v for v in vars_list if vars_list.count(v) > 1]
        if internal_dupes:
            print(f"\nRépétitions internes dans {group}:")
            print(set(internal_dupes))
        
        # Vérifier les répétitions avec d'autres groupes
        for v in vars_list:
            if v in classified_vars:
                duplicates[v] = duplicates.get(v, []) + [group]
            classified_vars.add(v)
    
    # 3. Identifier les variables non classées
    unclassified = all_vars - classified_vars
    
    # 4. Afficher les résultats
    print("\n=== Résultats ===")
    print(f"Total variables dans DataFrame: {len(all_vars)}")
    print(f"Total variables classifiées: {len(classified_vars)}")
    print(f"Variables non classées: {len(unclassified)}")
    
    if duplicates:
        print("\nVariables présentes dans plusieurs groupes:")
        for var, groups in duplicates.items():
            print(f"- {var}: {groups}")
    
    if unclassified:
        print("\nVariables non classées:")
        print(sorted(unclassified))
    
    return {
        'group_stats': {group: len(vars_) for group, vars_ in group_vars.items()},
        'duplicates': duplicates,
        'unclassified': unclassified
    }

# Exécuter la vérification
classification_results = verify_variable_classification()

In [4]:
# Cellule 1 : Fonction utilitaire
def flatten_vars(var_dict):
    """
    Aplatit un dictionnaire imbriqué de variables en une liste.
    """
    flat_list = []
    for k, v in var_dict.items():
        if isinstance(v, dict):
            for sub_v in v.values():
                if isinstance(sub_v, list):
                    flat_list.extend(sub_v)
                elif isinstance(sub_v, dict):
                    flat_list.extend(sum(sub_v.values(), []))
        elif isinstance(v, list):
            flat_list.extend(v)
    return flat_list

In [5]:
# Cellule 2 : Analyse initiale
def analyze_missing_values(df):
    """
    Analyse détaillée des valeurs manquantes par groupe et par tranche.
    """
    groups = {
        'Weather': sum(WEATHER_VARS.values(), []),
        'Building': sum(BUILDING_VARS.values(), []),
        'Activity': flatten_vars(ACTIVITY_VARS),
        'Insurance': flatten_vars(INSURANCE_VARS),
        'Geographic': flatten_vars(GEOGRAPHIC_VARS),
        'Demographic': flatten_vars(DEMOGRAPHIC_VARS)
    }
    
    missing_summary = pd.DataFrame({
        'Missing Count': df.isnull().sum(),
        'Missing Percentage': df.isnull().sum() / len(df) * 100
    }).sort_values('Missing Percentage', ascending=False)
    
    print_missing_analysis(df, missing_summary, groups)
    
    return missing_summary, groups

def print_missing_analysis(df, missing_summary, groups):
    """Affiche l'analyse des valeurs manquantes"""
    ranges = {
        '0%': (0, 0),
        '< 10%': (0, 10),
        '10-30%': (10, 30),
        '30-50%': (30, 50),
        '50-70%': (50, 70),
        '70-90%': (70, 90),
        '> 90%': (90, 100)
    }
    
    total_cols = 0
    print("=== Analyse des valeurs manquantes par tranche ===\n")
    
    for range_name, (min_val, max_val) in ranges.items():
        range_cols = missing_summary[
            missing_summary['Missing Percentage'].between(min_val, max_val, inclusive='right' if range_name != '0%' else 'both')
        ]
        
        n_cols = len(range_cols)
        total_cols += n_cols
        
        if n_cols > 0:
            print(f"\n=== Tranche {range_name} ({n_cols} colonnes) ===")
            for group_name, cols in groups.items():
                group_cols = [col for col in range_cols.index if col in cols]
                if group_cols:
                    print(f"\n{group_name}: {len(group_cols)} colonnes")
                    print(f"Variables: {group_cols}")
    
    print(f"\nTotal des colonnes analysées: {total_cols}")
    print(f"Nombre total de colonnes: {len(df.columns)}")

In [6]:
# Cellule 3 : Préparation des données
exclude_cols = ['FREQ', 'CM', 'CHARGE']
X_train = train_df.drop(exclude_cols, axis=1)
y_train = train_df[['FREQ', 'CM', 'CHARGE']]
X_test = test_df.copy()

# Analyse initiale
missing_summary, groups = analyze_missing_values(X_train)

=== Analyse des valeurs manquantes par tranche ===


=== Tranche 0% (96 colonnes) ===

Building: 30 colonnes
Variables: ['ADOSS', 'CARACT4', 'CARACT5', 'TYPBAT2', 'NBBAT14', 'NBBAT4', 'NBBAT5', 'NBBAT6', 'NBBAT7', 'NBBAT8', 'NBBAT9', 'NBBAT10', 'NBBAT11', 'NBBAT13', 'NBBAT3', 'NBBAT2', 'NBBAT1', 'SURFACE21', 'SURFACE1', 'SURFACE2', 'SURFACE4', 'SURFACE5', 'SURFACE6', 'SURFACE12', 'SURFACE13', 'SURFACE14', 'SURFACE15', 'SURFACE16', 'SURFACE17', 'SURFACE20']

Activity: 9 colonnes
Variables: ['ACTIVIT2', 'VOCATION', 'TYPERS', 'EQUIPEMENT7', 'COEFASS', 'EQUIPEMENT1', 'EQUIPEMENT3', 'EQUIPEMENT4', 'EQUIPEMENT6']

Insurance: 55 colonnes
Variables: ['AN_EXERC', 'KAPITAL7', 'CA1', 'CA2', 'CA3', 'KAPITAL1', 'KAPITAL2', 'KAPITAL3', 'KAPITAL4', 'KAPITAL5', 'KAPITAL6', 'KAPITAL8', 'KAPITAL34', 'KAPITAL9', 'KAPITAL10', 'KAPITAL12', 'KAPITAL13', 'KAPITAL14', 'KAPITAL25', 'KAPITAL26', 'KAPITAL27', 'KAPITAL28', 'TAILLE2', 'TAILLE1', 'DEROG15', 'DEROG11', 'ANCIENNETE', 'INDEM1', 'FRCH1', 'DEROG1', 'DER

In [ ]:
# Cellule 4 : Traitement Weather
def clean_weather_value(value):
    """
    Nettoie une valeur météo en extrayant le nombre après '<=' ou '>='
    Ex: '02. <= 65' -> 65
    Ex: '04. >= 7' -> 7
    """
    if pd.isna(value):
        return value
    try:
        # Enlever le préfixe (ex: "02. ")
        value = value.split('.')[1].strip()
        # Extraire le nombre après '<=' ou '>='
        if '<=' in value:
            return float(value.split('<=')[1].strip())
        elif '>=' in value:
            return float(value.split('>=')[1].strip())
        return value
    except:
        return value

def  clean_weather_vars(X_train, X_test, missing_summary, weather_cols):
    """
    Traite les variables météo avec conversion des valeurs catégorielles en numériques
    """
    missing_indicators = {}
    print("\nTraitement des variables météo:")
    
    for col in weather_cols:
        if col in X_train.columns:
            missing_pct = missing_summary.loc[col, 'Missing Percentage']
            print(f"\nColonne {col}:")
            print(f"- Pourcentage de valeurs manquantes: {missing_pct:.2f}%")
            print(f"- Type initial: {X_train[col].dtype}")
            
            # Afficher quelques valeurs avant conversion
            if X_train[col].dtype == 'object':
                print("- Exemples de valeurs avant conversion:")
                print(X_train[col].dropna().unique()[:5])
            
            # Créer indicateur de valeurs manquantes
            if missing_pct > 0:
                missing_indicators[f'{col}_is_missing'] = X_train[col].isnull().astype(int)
            
            if missing_pct <= 70:  # On traite uniquement si moins de 70% manquant
                # Convertir les valeurs catégorielles en numériques
                if X_train[col].dtype == 'object':
                    print("- Conversion en numérique")
                    X_train[col] = X_train[col].apply(clean_weather_value)
                    X_test[col] = X_test[col].apply(clean_weather_value)
                    print(f"- Type après conversion: {X_train[col].dtype}")
                
                # Imputation avec la médiane
                median_val = X_train[col].median()
                X_train[col].fillna(median_val, inplace=True)
                X_test[col].fillna(median_val, inplace=True)
                print(f"- Imputation avec la médiane: {median_val}")
                
                # Vérification des valeurs uniques après traitement
                unique_vals = sorted(X_train[col].unique())[:5]
                print(f"- Exemples de valeurs après traitement: {unique_vals}")
            else:
                print("- Colonne ignorée (>70% manquant)")
    
    return X_train, X_test, missing_indicators

# Exécution
X_train, X_test, weather_indicators = handle_weather_vars(
    X_train, X_test, missing_summary, groups['Weather']
)


Traitement des variables météo:

Colonne NBJTX25_MM_A:
- Pourcentage de valeurs manquantes: 56.75%
- Type initial: object
- Exemples de valeurs avant conversion:
[5.0 7.0 '04. >= 7' 4.0]
- Conversion en numérique
- Type après conversion: float64
- Imputation avec la médiane: 5.0
- Exemples de valeurs après traitement: [4.0, 5.0, 7.0]

Colonne NBJTX25_MMAX_A:
- Pourcentage de valeurs manquantes: 56.75%
- Type initial: object
- Exemples de valeurs avant conversion:
['03. <= 23' '04. >= 23' '02. <= 18' '01. <= 12']
- Conversion en numérique
- Type après conversion: float64
- Imputation avec la médiane: 23.0
- Exemples de valeurs après traitement: [12.0, 18.0, 23.0]

Colonne NBJTX25_MSOM_A:
- Pourcentage de valeurs manquantes: 56.75%
- Type initial: object
- Exemples de valeurs avant conversion:
['02. <= 65' '03. <= 88' '04. >= 88' '01. <= 44']
- Conversion en numérique
- Type après conversion: float64
- Imputation avec la médiane: 65.0
- Exemples de valeurs après traitement: [44.0, 65.0,

In [ ]:
# Cellule 5 : Traitement Building
def handle_building_vars(X_train, X_test, missing_summary, building_cols):
    """
    Traite les variables bâtiment avec vérification détaillée des types
    """
    missing_indicators = {}
    print("\nTraitement des variables bâtiment:")
    
    for col in building_cols:
        if col in X_train.columns:
            missing_pct = missing_summary.loc[col, 'Missing Percentage']
            print(f"\nColonne {col}:")
            print(f"- Pourcentage de valeurs manquantes: {missing_pct:.2f}%")
            print(f"- Type initial: {X_train[col].dtype}")
            
            # Créer indicateur de valeurs manquantes
            if missing_pct > 0:
                missing_indicators[f'{col}_is_missing'] = X_train[col].isnull().astype(int)
            
            if missing_pct <= 70:  # On traite uniquement si moins de 70% manquant
                if X_train[col].dtype in ['int64', 'float64']:
                    # Pour les variables numériques
                    median_val = X_train[col].median()
                    X_train[col].fillna(median_val, inplace=True)
                    X_test[col].fillna(median_val, inplace=True)
                    print(f"- Variable numérique, imputation avec la médiane: {median_val}")
                    print(f"- Plage de valeurs: [{X_train[col].min()}, {X_train[col].max()}]")
                else:
                    # Pour les variables catégorielles
                    # Filtrer les valeurs non-nulles avant de les afficher
                    unique_vals = X_train[col].dropna().unique()
                    print("- Variable catégorielle, valeurs uniques:")
                    print(unique_vals[:5])
                    
                    # Si peu de valeurs uniques, on pourrait considérer une conversion en numérique
                    if len(unique_vals) < 10:
                        print(f"- Nombre de catégories: {len(unique_vals)}")
                        print("- Distribution des valeurs:")
                        print(X_train[col].value_counts().head())
                    
                    mode_val = X_train[col].mode()[0]
                    X_train[col].fillna('UNKNOWN', inplace=True)
                    X_test[col].fillna('UNKNOWN', inplace=True)
                    print(f"- Imputation avec 'UNKNOWN'")
            else:
                print("- Colonne ignorée (>70% manquant)")
            
            # Vérification après traitement
            missing_after = X_train[col].isnull().sum()
            if missing_after > 0:
                print(f"⚠️ ATTENTION: {missing_after} valeurs manquantes restantes!")
    
    return X_train, X_test, missing_indicators

# Exécution
X_train, X_test, building_indicators = handle_building_vars(
    X_train, X_test, missing_summary, groups['Building']
)

In [ ]:
# Cellule 6 : Traitement Activity
def handle_activity_vars(X_train, X_test, missing_summary, activity_cols):
    """
    Traite les variables d'activité avec vérification détaillée des types et valeurs
    """
    missing_indicators = {}
    print("\nTraitement des variables d'activité:")
    
    for col in activity_cols:
        if col in X_train.columns:
            missing_pct = missing_summary.loc[col, 'Missing Percentage']
            print(f"\nColonne {col}:")
            print(f"- Pourcentage de valeurs manquantes: {missing_pct:.2f}%")
            print(f"- Type initial: {X_train[col].dtype}")
            
            # Créer indicateur de valeurs manquantes
            if missing_pct > 0:
                missing_indicators[f'{col}_is_missing'] = X_train[col].isnull().astype(int)
            
            if missing_pct <= 70:  # On traite uniquement si moins de 70% manquant
                if X_train[col].dtype in ['int64', 'float64']:
                    # Pour les variables numériques
                    median_val = X_train[col].median()
                    X_train[col].fillna(median_val, inplace=True)
                    X_test[col].fillna(median_val, inplace=True)
                    print(f"- Variable numérique, imputation avec la médiane: {median_val}")
                    print(f"- Plage de valeurs: [{X_train[col].min()}, {X_train[col].max()}]")
                    print(f"- Distribution: \n{X_train[col].value_counts().head()}")
                else:
                    # Pour les variables catégorielles
                    unique_vals = X_train[col].unique()
                    print(f"- Variable catégorielle")
                    print(f"- Nombre de catégories uniques: {len(unique_vals)}")
                    print(f"- Top 5 catégories les plus fréquentes:")
                    print(X_train[col].value_counts().head())
                    
                    mode_val = X_train[col].mode()[0]
                    X_train[col].fillna('UNKNOWN', inplace=True)
                    X_test[col].fillna('UNKNOWN', inplace=True)
                    print(f"- Imputation avec 'UNKNOWN'")
            else:
                print("- Colonne ignorée (>70% manquant)")
            
            # Vérification après traitement
            missing_after = X_train[col].isnull().sum()
            if missing_after > 0:
                print(f"⚠️ ATTENTION: {missing_after} valeurs manquantes restantes!")
            
            # Vérification de la cohérence entre train et test
            train_unique = set(X_train[col].unique())
            test_unique = set(X_test[col].unique())
            diff_cats = test_unique - train_unique
            if len(diff_cats) > 0:
                print(f"⚠️ ATTENTION: Catégories présentes dans test mais pas dans train: {diff_cats}")
    
    return X_train, X_test, missing_indicators

# Exécution
X_train, X_test, activity_indicators = handle_activity_vars(
    X_train, X_test, missing_summary, groups['Activity']
)

In [12]:
# Cellule 7 : Traitement Insurance
def handle_insurance_vars(X_train, X_test, missing_summary, insurance_cols):
    """
    Traite les variables d'assurance avec vérification détaillée des types et valeurs
    """
    missing_indicators = {}
    print("\nTraitement des variables d'assurance:")
    
    for col in insurance_cols:
        if col in X_train.columns:
            missing_pct = missing_summary.loc[col, 'Missing Percentage']
            print(f"\nColonne {col}:")
            print(f"- Pourcentage de valeurs manquantes: {missing_pct:.2f}%")
            print(f"- Type initial: {X_train[col].dtype}")
            
            # Créer indicateur de valeurs manquantes
            if missing_pct > 0:
                missing_indicators[f'{col}_is_missing'] = X_train[col].isnull().astype(int)
            
            if missing_pct <= 70:  # On traite uniquement si moins de 70% manquant
                if X_train[col].dtype in ['int64', 'float64']:
                    # Pour les variables numériques
                    median_val = X_train[col].median()
                    mean_val = X_train[col].mean()
                    std_val = X_train[col].std()
                    
                    print(f"- Variable numérique:")
                    print(f"  * Médiane: {median_val}")
                    print(f"  * Moyenne: {mean_val:.2f}")
                    print(f"  * Écart-type: {std_val:.2f}")
                    print(f"  * Plage: [{X_train[col].min()}, {X_train[col].max()}]")
                    
                    # Détection des valeurs aberrantes
                    outliers = X_train[col][(X_train[col] > mean_val + 3*std_val) | 
                                         (X_train[col] < mean_val - 3*std_val)].count()
                    if outliers > 0:
                        print(f"⚠️ {outliers} valeurs aberrantes détectées (>3σ)")
                    
                    X_train[col].fillna(median_val, inplace=True)
                    X_test[col].fillna(median_val, inplace=True)
                    
                else:
                    # Pour les variables catégorielles
                    unique_vals = X_train[col].unique()
                    print(f"- Variable catégorielle:")
                    print(f"  * Nombre de catégories: {len(unique_vals)}")
                    print("  * Distribution des valeurs:")
                    print(X_train[col].value_counts().head())
                    
                    # Si la variable a peu de catégories, on pourrait envisager un encodage
                    if len(unique_vals) < 10:
                        print("  * Candidat potentiel pour encodage catégoriel")
                    
                    X_train[col].fillna('UNKNOWN', inplace=True)
                    X_test[col].fillna('UNKNOWN', inplace=True)
            else:
                print("- Colonne ignorée (>70% manquant)")
            
            # Vérifications post-traitement
            missing_after = X_train[col].isnull().sum()
            if missing_after > 0:
                print(f"⚠️ ATTENTION: {missing_after} valeurs manquantes restantes!")
            
            # Vérification de la cohérence train/test
            if X_train[col].dtype == 'object':
                train_cats = set(X_train[col].unique())
                test_cats = set(X_test[col].unique())
                new_cats = test_cats - train_cats
                if new_cats:
                    print(f"⚠️ Nouvelles catégories dans test: {new_cats}")
    
    return X_train, X_test, missing_indicators

# Exécution
X_train, X_test, insurance_indicators = handle_insurance_vars(
    X_train, X_test, missing_summary, groups['Insurance']
)


Traitement des variables d'assurance:

Colonne RISK1:
- Pourcentage de valeurs manquantes: 6.93%
- Type initial: float64
- Variable numérique:
  * Médiane: 3.0
  * Moyenne: 447.41
  * Écart-type: 495.72
  * Plage: [-1.0, 1000.0]

Colonne RISK2:
- Pourcentage de valeurs manquantes: 6.93%
- Type initial: float64
- Variable numérique:
  * Médiane: 35.0
  * Moyenne: 461.45
  * Écart-type: 483.14
  * Plage: [3.0, 1000.0]

Colonne RISK3:
- Pourcentage de valeurs manquantes: 6.93%
- Type initial: float64
- Variable numérique:
  * Médiane: 13.0
  * Moyenne: 451.24
  * Écart-type: 492.29
  * Plage: [-3.0, 1000.0]

Colonne RISK4:
- Pourcentage de valeurs manquantes: 6.93%
- Type initial: float64
- Variable numérique:
  * Médiane: 5.0
  * Moyenne: 447.76
  * Écart-type: 495.40
  * Plage: [1.0, 1000.0]

Colonne RISK5:
- Pourcentage de valeurs manquantes: 6.93%
- Type initial: float64
- Variable numérique:
  * Médiane: 5.0
  * Moyenne: 448.33
  * Écart-type: 494.89
  * Plage: [0.0, 1000.0]

Colonn

In [19]:
def handle_geographic_vars(X_train, X_test, missing_summary, geographic_cols):
    """
    Traite les variables géographiques avec vérification détaillée des types et valeurs
    """
    missing_indicators = {}
    print("\nTraitement des variables géographiques:")
    
    # Regrouper les colonnes par type
    distance_cols = [col for col in geographic_cols if 'DISTANCE' in col]
    proportion_cols = [col for col in geographic_cols if 'PROPORTION' in col]
    altitude_cols = [col for col in geographic_cols if 'ALTITUDE' in col]
    other_cols = [col for col in geographic_cols if col not in distance_cols + proportion_cols + altitude_cols]
    
    for col_type, cols in [
        ("Distance", distance_cols),
        ("Proportion", proportion_cols),
        ("Altitude", altitude_cols),
        ("Autres", other_cols)
    ]:
        print(f"\n=== Variables de {col_type} ===")
        
        for col in cols:
            if col in X_train.columns:
                missing_pct = missing_summary.loc[col, 'Missing Percentage']
                print(f"\nColonne {col}:")
                print(f"- Pourcentage de valeurs manquantes: {missing_pct:.2f}%")
                print(f"- Type initial: {X_train[col].dtype}")
                
                # Créer indicateur de valeurs manquantes
                if missing_pct > 0:
                    missing_indicators[f'{col}_is_missing'] = X_train[col].isnull().astype(int)
                
                if missing_pct <= 70:  # On traite uniquement si moins de 70% manquant
                    if X_train[col].dtype in ['int64', 'float64']:
                        # Pour les variables numériques
                        stats = X_train[col].describe()
                        print("- Statistiques descriptives:")
                        print(f"  * Moyenne: {stats['mean']:.2f}")
                        print(f"  * Écart-type: {stats['std']:.2f}")
                        print(f"  * Min: {stats['min']:.2f}")
                        print(f"  * 25%: {stats['25%']:.2f}")
                        print(f"  * Médiane: {stats['50%']:.2f}")
                        print(f"  * 75%: {stats['75%']:.2f}")
                        print(f"  * Max: {stats['max']:.2f}")
                        
                        # Vérification des valeurs négatives pour les distances
                        if 'DISTANCE' in col and (X_train[col] < 0).any():
                            print("⚠️ ATTENTION: Valeurs négatives détectées pour une distance!")
                        
                        # Vérification des proportions
                        if 'PROPORTION' in col:
                            if (X_train[col] < 0).any() or (X_train[col] > 1).any():
                                print("⚠️ ATTENTION: Proportions hors de l'intervalle [0,1]!")
                        
                        median_val = stats['50%']
                        X_train[col].fillna(median_val, inplace=True)
                        X_test[col].fillna(median_val, inplace=True)
                        print(f"- Imputation avec la médiane: {median_val}")
                        
                    else:
                        # Pour les variables catégorielles
                        # Extraire le nombre après '<=' ou '>='
                        def extract_number(x):
                            if pd.isna(x):
                                return x
                            try:
                                if '<=' in str(x):
                                    return float(str(x).split('<=')[1].strip())
                                elif '>=' in str(x):
                                    return float(str(x).split('>=')[1].strip())
                                return float(x)
                            except:
                                return x
                        
                        print("- Conversion en numérique...")
                        X_train[col] = X_train[col].apply(extract_number)
                        X_test[col] = X_test[col].apply(extract_number)
                        
                        # Imputation avec la médiane après conversion
                        median_val = X_train[col].median()
                        X_train[col].fillna(median_val, inplace=True)
                        X_test[col].fillna(median_val, inplace=True)
                        print(f"- Imputation avec la médiane: {median_val}")
                        
                else:
                    print("- Colonne ignorée (>70% manquant)")
                
                # Vérifications post-traitement
                missing_after = X_train[col].isnull().sum()
                if missing_after > 0:
                    print(f"⚠️ ATTENTION: {missing_after} valeurs manquantes restantes!")
    
    return X_train, X_test, missing_indicators

In [15]:
# Cellule 9 : Traitement Demographic
def handle_demographic_vars(X_train, X_test, missing_summary, demographic_cols):
    """
    Traite les variables démographiques avec vérification détaillée des types et valeurs
    """
    missing_indicators = {}
    print("\nTraitement des variables démographiques:")
    
    # Regrouper les colonnes par type
    household_cols = [col for col in demographic_cols if col.startswith('MEN')]
    individual_cols = [col for col in demographic_cols if col.startswith('IND')]
    housing_cols = [col for col in demographic_cols if col.startswith('LOG')]
    
    for col_type, cols in [
        ("Ménages", household_cols),
        ("Individus", individual_cols),
        ("Logements", housing_cols)
    ]:
        print(f"\n=== Variables {col_type} ===")
        
        for col in cols:
            if col in X_train.columns:
                missing_pct = missing_summary.loc[col, 'Missing Percentage']
                print(f"\nColonne {col}:")
                print(f"- Pourcentage de valeurs manquantes: {missing_pct:.2f}%")
                print(f"- Type initial: {X_train[col].dtype}")
                
                # Créer indicateur de valeurs manquantes
                if missing_pct > 0:
                    missing_indicators[f'{col}_is_missing'] = X_train[col].isnull().astype(int)
                
                if missing_pct <= 70:  # On traite uniquement si moins de 70% manquant
                    if X_train[col].dtype in ['int64', 'float64']:
                        # Pour les variables numériques
                        stats = X_train[col].describe()
                        print("- Statistiques descriptives:")
                        print(f"  * Moyenne: {stats['mean']:.2f}")
                        print(f"  * Écart-type: {stats['std']:.2f}")
                        print(f"  * Min: {stats['min']:.2f}")
                        print(f"  * 25%: {stats['25%']:.2f}")
                        print(f"  * Médiane: {stats['50%']:.2f}")
                        print(f"  * 75%: {stats['75%']:.2f}")
                        print(f"  * Max: {stats['max']:.2f}")
                        
                        # Vérifications spécifiques selon le type de variable
                        if col.startswith('MEN') or col.startswith('IND'):
                            if (X_train[col] < 0).any():
                                print("⚠️ ATTENTION: Valeurs négatives détectées pour un comptage!")
                        
                        # Vérification de la cohérence des proportions
                        if 'PROP' in col or 'PAUV' in col:
                            if (X_train[col] < 0).any() or (X_train[col] > 1).any():
                                print("⚠️ ATTENTION: Proportions hors de l'intervalle [0,1]!")
                        
                        median_val = stats['50%']
                        X_train[col].fillna(median_val, inplace=True)
                        X_test[col].fillna(median_val, inplace=True)
                        print(f"- Imputation avec la médiane: {median_val}")
                        
                    else:
                        # Pour les variables catégorielles
                        value_counts = X_train[col].value_counts()
                        print("- Distribution des valeurs:")
                        print(value_counts.head())
                        print(f"- Nombre total de catégories: {len(value_counts)}")
                        
                        X_train[col].fillna('UNKNOWN', inplace=True)
                        X_test[col].fillna('UNKNOWN', inplace=True)
                        print("- Imputation avec 'UNKNOWN'")
                else:
                    print("- Colonne ignorée (>70% manquant)")
                
                # Vérifications post-traitement
                missing_after = X_train[col].isnull().sum()
                if missing_after > 0:
                    print(f"⚠️ ATTENTION: {missing_after} valeurs manquantes restantes!")
                
                # Vérification de la cohérence des données
                if col.startswith('MEN'):
                    total_men = X_train['MEN'] if 'MEN' in X_train.columns else None
                    if total_men is not None and col != 'MEN':
                        if (X_train[col] > total_men).any():
                            print("⚠️ ATTENTION: Sous-catégorie de ménages supérieure au total!")
                
                if col.startswith('IND'):
                    total_ind = X_train['IND'] if 'IND' in X_train.columns else None
                    if total_ind is not None and col != 'IND':
                        if (X_train[col] > total_ind).any():
                            print("⚠️ ATTENTION: Sous-catégorie d'individus supérieure au total!")
    
    return X_train, X_test, missing_indicators

# Exécution
X_train, X_test, demographic_indicators = handle_demographic_vars(
    X_train, X_test, missing_summary, groups['Demographic']
)


Traitement des variables démographiques:

=== Variables Ménages ===

Colonne MEN:
- Pourcentage de valeurs manquantes: 4.85%
- Type initial: object
- Distribution des valeurs:
01. <= 17204     362312
02. <= 153098      2592
03. <= 670263       116
Name: MEN, dtype: int64
- Nombre total de catégories: 3
- Imputation avec 'UNKNOWN'

Colonne MEN_1IND:
- Pourcentage de valeurs manquantes: 4.85%
- Type initial: object
- Distribution des valeurs:
03. <= 30    160610
04. <= 40    147462
05. <= 50     37410
02. <= 20     18298
06. <= 60      1221
Name: MEN_1IND, dtype: int64
- Nombre total de catégories: 7
- Imputation avec 'UNKNOWN'
⚠️ ATTENTION: Sous-catégorie de ménages supérieure au total!

Colonne MEN_5IND:
- Pourcentage de valeurs manquantes: 4.85%
- Type initial: object
- Distribution des valeurs:
01. <= 10    350980
02. <= 20     14015
03. <= 30        25
Name: MEN_5IND, dtype: int64
- Nombre total de catégories: 3
- Imputation avec 'UNKNOWN'
⚠️ ATTENTION: Sous-catégorie de ménages su

In [20]:
# Cellule 10 : Finalisation
def finalize_processing(X_train, X_test, all_indicators, missing_summary):
    """
    Finalise le traitement des données avec vérifications détaillées
    """
    print("\n=== Début de la finalisation ===")
    
    # État initial
    print("\nÉtat initial:")
    print(f"X_train: {X_train.shape}")
    print(f"X_test: {X_test.shape}")
    print(f"Nombre d'indicateurs: {len(all_indicators)}")
    
    # Vérification des index
    print("\nVérification des index:")
    print(f"Index train unique: {X_train.index.is_unique}")
    print(f"Index test unique: {X_test.index.is_unique}")
    
    # Combiner les indicateurs
    print("\nCréation des DataFrames d'indicateurs...")
    indicators_df_train = pd.DataFrame(all_indicators, index=X_train.index)
    indicators_df_test = pd.DataFrame(all_indicators, index=X_test.index)
    
    print(f"Shape indicateurs train: {indicators_df_train.shape}")
    print(f"Shape indicateurs test: {indicators_df_test.shape}")
    
    # Concaténation
    print("\nConcaténation avec les données principales...")
    X_train = pd.concat([X_train, indicators_df_train], axis=1)
    X_test = pd.concat([X_test, indicators_df_test], axis=1)
    
    # Suppression des colonnes avec beaucoup de valeurs manquantes
    high_missing = missing_summary[missing_summary['Missing Percentage'] > 70].index
    print(f"\nSuppression de {len(high_missing)} colonnes avec >70% de valeurs manquantes:")
    print(high_missing.tolist())
    
    X_train.drop(high_missing, axis=1, inplace=True)
    X_test.drop(high_missing, axis=1, inplace=True)
    
    # Vérifications finales
    print("\n=== État final des données ===")
    print(f"Shape après traitement:")
    print(f"X_train: {X_train.shape}")
    print(f"X_test: {X_test.shape}")
    
    # Vérification des valeurs manquantes
    final_missing_train = X_train.isnull().sum()
    final_missing_test = X_test.isnull().sum()
    
    if final_missing_train.sum() > 0:
        print("\nColonnes avec valeurs manquantes restantes dans train:")
        print(final_missing_train[final_missing_train > 0])
    else:
        print("\nAucune valeur manquante restante dans train")
        
    if final_missing_test.sum() > 0:
        print("\nColonnes avec valeurs manquantes restantes dans test:")
        print(final_missing_test[final_missing_test > 0])
    else:
        print("\nAucune valeur manquante restante dans test")
    
    # Vérification des types de données
    print("\nTypes de données dans le jeu final:")
    print(X_train.dtypes.value_counts())
    
    # Vérification de la cohérence train/test
    train_cols = set(X_train.columns)
    test_cols = set(X_test.columns)
    
    if train_cols != test_cols:
        print("\n⚠️ ATTENTION: Différences dans les colonnes train/test!")
        print("Colonnes uniquement dans train:", train_cols - test_cols)
        print("Colonnes uniquement dans test:", test_cols - train_cols)
    else:
        print("\nColonnes cohérentes entre train et test")
    
    return X_train, X_test

# Initialiser le dictionnaire des indicateurs
all_indicators = {}

# Liste des groupes d'indicateurs à vérifier
indicator_groups = {
    'weather': ('weather_indicators', weather_indicators if 'weather_indicators' in globals() else {}),
    'building': ('building_indicators', building_indicators if 'building_indicators' in globals() else {}),
    'activity': ('activity_indicators', activity_indicators if 'activity_indicators' in globals() else {}),
    'insurance': ('insurance_indicators', insurance_indicators if 'insurance_indicators' in globals() else {}),
    'geographic': ('geographic_indicators', geographic_indicators if 'geographic_indicators' in globals() else {}),
    'demographic': ('demographic_indicators', demographic_indicators if 'demographic_indicators' in globals() else {})
}

# Ajouter chaque groupe d'indicateurs disponible
for group_name, (var_name, indicators) in indicator_groups.items():
    if indicators:
        print(f"Ajout de {len(indicators)} indicateurs du groupe {group_name}")
        all_indicators.update(indicators)
    else:
        print(f"⚠️ Attention: Les indicateurs {group_name} sont manquants ou vides")

# Traitement final
X_train_processed, X_test_processed = finalize_processing(
    X_train, X_test, all_indicators, missing_summary
)

Ajout de 130 indicateurs du groupe weather
Ajout de 16 indicateurs du groupe building
Ajout de 2 indicateurs du groupe activity
Ajout de 34 indicateurs du groupe insurance
⚠️ Attention: Les indicateurs geographic sont manquants ou vides
Ajout de 28 indicateurs du groupe demographic

=== Début de la finalisation ===

État initial:
X_train: (383610, 374)
X_test: (95852, 374)
Nombre d'indicateurs: 210

Vérification des index:
Index train unique: True
Index test unique: True

Création des DataFrames d'indicateurs...
Shape indicateurs train: (383610, 210)
Shape indicateurs test: (95852, 210)

Concaténation avec les données principales...

Suppression de 7 colonnes avec >70% de valeurs manquantes:
['DEROG14', 'DEROG13', 'DEROG16', 'CARACT2', 'CARACT3', 'TYPBAT1', 'DEROG12']

=== État final des données ===
Shape après traitement:
X_train: (383610, 577)
X_test: (95852, 577)

Colonnes avec valeurs manquantes restantes dans train:
DISTANCE_112    217712
DISTANCE_121    217712
DISTANCE_122    217

def flatten_vars(var_dict):
    """
    Aplatit un dictionnaire imbriqué de variables en une liste.
    """
    flat_list = []
    for k, v in var_dict.items():
        if isinstance(v, dict):
            for sub_v in v.values():
                if isinstance(sub_v, list):
                    flat_list.extend(sub_v)
                elif isinstance(sub_v, dict):
                    flat_list.extend(sum(sub_v.values(), []))
        elif isinstance(v, list):
            flat_list.extend(v)
    return flat_list

def analyze_missing_values(df):
    """
    Analyse détaillée des valeurs manquantes par groupe et par tranche.
    """
    # Définir les groupes de variables
    groups = {
        'Weather': sum(WEATHER_VARS.values(), []),
        'Building': sum(BUILDING_VARS.values(), []),
        'Insurance': flatten_vars(INSURANCE_VARS),
        'Activity': flatten_vars(ACTIVITY_VARS), 
        'Geographic': flatten_vars(GEOGRAPHIC_VARS),
        'Demographic': flatten_vars(DEMOGRAPHIC_VARS)
    }
    
    missing_summary = pd.DataFrame({
        'Missing Count': df.isnull().sum(),
        'Missing Percentage': df.isnull().sum() / len(df) * 100
    }).sort_values('Missing Percentage', ascending=False)
    
    # Définir les tranches
    ranges = {
        '0%': (0, 0),
        '< 10%': (0, 10),
        '10-30%': (10, 30),
        '30-50%': (30, 50),
        '50-70%': (50, 70),
        '70-90%': (70, 90),
        '> 90%': (90, 100)
    }
    
    # Analyser par tranche et par groupe
    total_cols = 0
    print("=== Analyse des valeurs manquantes par tranche ===\n")
    
    for range_name, (min_val, max_val) in ranges.items():
        if range_name == '0%':
            range_cols = missing_summary[missing_summary['Missing Percentage'] == 0]
        else:
            range_cols = missing_summary[
                (missing_summary['Missing Percentage'] > min_val) & 
                (missing_summary['Missing Percentage'] <= max_val)
            ]
        
        n_cols = len(range_cols)
        total_cols += n_cols
        
        if n_cols > 0:
            print(f"\n=== Tranche {range_name} ({n_cols} colonnes) ===")
            # Analyser par groupe
            for group_name, cols in groups.items():
                group_cols = [col for col in range_cols.index if col in cols]
                if group_cols:
                    print(f"\n{group_name}: {len(group_cols)} colonnes")
                    print(f"Variables: {group_cols}")
    
    print(f"\nTotal des colonnes analysées: {total_cols}")
    print(f"Nombre total de colonnes: {len(df.columns)}")
    
    return missing_summary

def handle_missing_values(X_train, X_test):
    """
    Traite les valeurs manquantes avec une approche optimisée.
    """
    print("=== État initial des données ===")
    print(f"X_train shape: {X_train.shape}")
    print(f"X_test shape: {X_test.shape}")
    
    # Analyse initiale
    missing_summary = analyze_missing_values(X_train)
    
    # Identifier les types de colonnes
    numeric_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
    categorical_cols = X_train.select_dtypes(include=['object']).columns
    
    # Définir les groupes de variables
    groups = {
        'Weather': sum(WEATHER_VARS.values(), []),
        'Building': sum(BUILDING_VARS.values(), []),
        'Activity': flatten_vars(ACTIVITY_VARS), 
        'Insurance': flatten_vars(INSURANCE_VARS),
        'Geographic': flatten_vars(GEOGRAPHIC_VARS),
        'Demographic': flatten_vars(DEMOGRAPHIC_VARS)
    }
    
    # Créer des DataFrames pour stocker les indicateurs de valeurs manquantes
    missing_indicators_train = pd.DataFrame()
    missing_indicators_test = pd.DataFrame()
    
    # Traitement par groupe
    print("\n=== Traitement par groupe ===")
    for group_name, cols in groups.items():
        present_cols = [col for col in cols if col in X_train.columns]
        print(f"\nTraitement {group_name}: {len(present_cols)} colonnes")
        
        for col in present_cols:
            missing_pct = missing_summary.loc[col, 'Missing Percentage']
            
            # Créer indicateur pour toutes les colonnes avec valeurs manquantes
            if missing_pct > 0:
                missing_indicators_train[f'{col}_is_missing'] = X_train[col].isnull().astype(int)
                missing_indicators_test[f'{col}_is_missing'] = X_test[col].isnull().astype(int)
            
            if missing_pct > 70:
                continue
                
            if col in numeric_cols:
                # Imputation simple avec la médiane
                median_value = X_train[col].median()
                X_train[col].fillna(median_value, inplace=True)
                X_test[col].fillna(median_value, inplace=True)
            
            elif col in categorical_cols:
                X_train[col].fillna('UNKNOWN', inplace=True)
                X_test[col].fillna('UNKNOWN', inplace=True)
    
    # Ajouter les indicateurs de valeurs manquantes
    X_train = pd.concat([X_train, missing_indicators_train], axis=1)
    X_test = pd.concat([X_test, missing_indicators_test], axis=1)
    
    # Supprimer les colonnes avec plus de 70% de valeurs manquantes
    high_missing = missing_summary[missing_summary['Missing Percentage'] > 70].index
    X_train.drop(high_missing, axis=1, inplace=True)
    X_test.drop(high_missing, axis=1, inplace=True)
    
    # Vérification finale
    print("\n=== État final des données ===")
    print(f"Shape après traitement:")
    print(f"X_train: {X_train.shape}")
    print(f"X_test: {X_test.shape}")
    
    final_missing = X_train.isnull().sum()
    if final_missing.sum() > 0:
        print("\nColonnes avec valeurs manquantes restantes:")
        print(final_missing[final_missing > 0])
    else:
        print("\nAucune valeur manquante restante")
    
    return X_train, X_test

# Application
exclude_cols = ['FREQ', 'CM', 'CHARGE']
X_train = train_df.drop(exclude_cols, axis=1)
y_train = train_df[['FREQ', 'CM', 'CHARGE']]
X_test = test_df.copy()

X_train_processed, X_test_processed = handle_missing_values(X_train, X_test)




def analyze_missing_values():
    """Analyse les valeurs manquantes par tranches et par groupes."""
    
    # 1. Calculer le pourcentage de valeurs manquantes pour chaque colonne
    missing_summary = pd.DataFrame({
        'Missing Count': train_df.isnull().sum(),
        'Missing Percentage': train_df.isnull().sum() / len(train_df) * 100
    }).sort_values('Missing Percentage', ascending=False)
    
    # 2. Définir les tranches de valeurs manquantes
    ranges = {
        '< 10%': (0, 10),
        '10-30%': (10, 30),
        '30-50%': (30, 50),
        '50-70%': (50, 70),
        '70-90%': (70, 90),
        '> 90%': (90, 100)
    }
    
    # 3. Fonction helper pour aplatir les dictionnaires imbriqués
    def flatten_vars(var_dict):
        flat_list = []
        for k, v in var_dict.items():
            if isinstance(v, dict):
                for sub_v in v.values():
                    if isinstance(sub_v, list):
                        flat_list.extend(sub_v)
                    elif isinstance(sub_v, dict):
                        flat_list.extend(sum(sub_v.values(), []))
            elif isinstance(v, list):
                flat_list.extend(v)
        return flat_list
    
    # 4. Analyser par groupe et par tranche
    print("=== Analyse des valeurs manquantes par groupe et par tranche ===\n")
    
    for range_name, (min_val, max_val) in ranges.items():
        print(f"\n=== Tranche {range_name} ===")
        
        # Filtrer les colonnes dans cette tranche
        range_missing = missing_summary[
            (missing_summary['Missing Percentage'] > min_val) & 
            (missing_summary['Missing Percentage'] <= max_val)
        ]
        
        if range_missing.empty:
            print("Aucune variable dans cette tranche")
            continue
        
        # Analyser par groupe
        missing_by_group = {
            'Weather': [col for col in range_missing.index if col in 
                       sum(WEATHER_VARS.values(), [])],
            'Building': [col for col in range_missing.index if col in 
                        flatten_vars(BUILDING_VARS)],
            'Insurance': [col for col in range_missing.index if col in 
                         flatten_vars(INSURANCE_VARS)],
            'Geographic': [col for col in range_missing.index if col in 
                          flatten_vars(GEOGRAPHIC_VARS)],
            'Demographic': [col for col in range_missing.index if col in 
                           flatten_vars(DEMOGRAPHIC_VARS)],
            'Activity': [col for col in range_missing.index if col in 
                        flatten_vars(ACTIVITY_VARS)],
            'Target': [col for col in range_missing.index if col in 
                      flatten_vars(TARGET_VARS)],
            'ID': [col for col in range_missing.index if col in 
                   ID_VARS['identifiers']],
            'Others': []
        }
        
        # Ajouter les colonnes non classées dans 'Others'
        all_grouped = sum(missing_by_group.values(), [])
        missing_by_group['Others'] = [col for col in range_missing.index 
                                    if col not in all_grouped]
        
        # Afficher les résultats pour cette tranche
        for group, cols in missing_by_group.items():
            if cols:  # N'afficher que les groupes non vides
                print(f"\n{group}: {len(cols)} colonnes")
                print(f"Pourcentage moyen de valeurs manquantes: "
                      f"{missing_summary.loc[cols, 'Missing Percentage'].mean():.2f}%")
                print(f"Variables: {cols}")
    
    # 5. Résumé global
    print("\n=== Résumé global ===")
    print("Distribution des valeurs manquantes par tranche:")
    for range_name, (min_val, max_val) in ranges.items():
        n_cols = len(missing_summary[
            (missing_summary['Missing Percentage'] > min_val) & 
            (missing_summary['Missing Percentage'] <= max_val)
        ])
        print(f"- {range_name}: {n_cols} colonnes")
    
    return missing_summary

# Exécuter l'analyse
missing_summary = analyze_missing_values()

def flatten_vars(var_dict):
    """
    Aplatit un dictionnaire imbriqué de variables en une liste.
    """
    flat_list = []
    for k, v in var_dict.items():
        if isinstance(v, dict):
            for sub_v in v.values():
                if isinstance(sub_v, list):
                    flat_list.extend(sub_v)
                elif isinstance(sub_v, dict):
                    flat_list.extend(sum(sub_v.values(), []))
        elif isinstance(v, list):
            flat_list.extend(v)
    return flat_list

def handle_missing_values(X_train, X_test):
    """
    Traite les valeurs manquantes avec vérifications détaillées à chaque étape.
    """
    print("=== État initial des données ===")
    print(f"X_train shape: {X_train.shape}")
    print(f"X_test shape: {X_test.shape}")
    
    # 1. Analyse initiale des valeurs manquantes
    print("\n=== Analyse initiale des valeurs manquantes ===")
    missing_pct = (X_train.isnull().sum() / len(X_train) * 100).sort_values(ascending=False)
    print("\nDistribution des valeurs manquantes:")
    for threshold in [90, 70, 50, 30, 10, 0]:
        n_cols = sum(missing_pct > threshold)
        print(f"> {threshold}%: {n_cols} colonnes")
    
    # 2. Identifier les types de colonnes
    numeric_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
    categorical_cols = X_train.select_dtypes(include=['object']).columns
    print(f"\nTypes de colonnes:")
    print(f"Numériques: {len(numeric_cols)}")
    print(f"Catégorielles: {len(categorical_cols)}")
    
    # 3. Traitement par groupe de variables
    groups = {
        'Weather': sum(WEATHER_VARS.values(), []),
        'Building': sum(BUILDING_VARS.values(), []),
        'Insurance': flatten_vars(INSURANCE_VARS),
        'Geographic': flatten_vars(GEOGRAPHIC_VARS),
        'Demographic': flatten_vars(DEMOGRAPHIC_VARS)
    }
    
    print("\n=== Traitement par groupe ===")
    for group_name, cols in groups.items():
        print(f"\nTraitement {group_name}:")
        present_cols = [col for col in cols if col in X_train.columns]
        print(f"Colonnes trouvées: {len(present_cols)}/{len(cols)}")
        
        for col in present_cols:
            if col in numeric_cols:
                # Créer indicateur de valeur manquante
                X_train[f'{col}_is_missing'] = X_train[col].isnull().astype(int)
                X_test[f'{col}_is_missing'] = X_test[col].isnull().astype(int)
                
                # Imputation selon le groupe
                if group_name == 'Weather':
                    group_col = 'ZONE'
                elif group_name == 'Building':
                    group_col = 'TYPBAT2'
                elif group_name == 'Insurance':
                    group_col = ['RISK1', 'RISK2']
                else:
                    group_col = 'ZONE'
                
                # Imputation avec vérification
                try:
                    if isinstance(group_col, list):
                        X_train[col] = X_train.groupby(group_col)[col].transform(
                            lambda x: x.fillna(x.median()))
                        X_test[col] = X_test.groupby(group_col)[col].transform(
                            lambda x: x.fillna(x.median()))
                    else:
                        X_train[col] = X_train.groupby(group_col)[col].transform(
                            lambda x: x.fillna(x.median()))
                        X_test[col] = X_test.groupby(group_col)[col].transform(
                            lambda x: x.fillna(x.median()))
                except Exception as e:
                    print(f"Erreur pour {col}: {str(e)}")
                    # Fallback à la médiane globale
                    X_train[col] = X_train[col].fillna(X_train[col].median())
                    X_test[col] = X_test[col].fillna(X_train[col].median())
            
            elif col in categorical_cols:
                X_train[col] = X_train[col].fillna('UNKNOWN')
                X_test[col] = X_test[col].fillna('UNKNOWN')
    
    # 4. Vérification des colonnes très manquantes
    print("\n=== Colonnes avec beaucoup de valeurs manquantes ===")
    high_missing_pct = (X_train.isnull().sum() / len(X_train) * 100)
    high_missing = high_missing_pct[high_missing_pct > 70].index.tolist()
    print(f"Colonnes >70% manquantes avant suppression:")
    for col in high_missing:
        print(f"{col}: {high_missing_pct[col]:.2f}%")
    
    # 5. Suppression des colonnes
    X_train = X_train.drop(high_missing, axis=1)
    X_test = X_test.drop(high_missing, axis=1)
    
    # 6. Vérification finale
    print("\n=== État final des données ===")
    print(f"Shape après traitement:")
    print(f"X_train: {X_train.shape}")
    print(f"X_test: {X_test.shape}")
    
    missing_final = X_train.isnull().sum()
    if missing_final.sum() > 0:
        print("\nValeurs manquantes restantes:")
        print(missing_final[missing_final > 0])
    else:
        print("\nAucune valeur manquante restante")
    
    return X_train, X_test

# Application
exclude_cols = ['FREQ', 'CM', 'CHARGE']
X_train = train_df.drop(exclude_cols, axis=1)
y_train = train_df[['FREQ', 'CM', 'CHARGE']]
X_test = test_df.copy()

X_train_processed, X_test_processed = handle_missing_values(X_train, X_test)

In [ ]:
def propose_strategies():
    print("=== Stratégies proposées par groupe ===\n")
    
    strategies = {
        'Weather': {
            'description': "Variables météorologiques (température, vent, pluie)",
            'options': [
                "Imputation par la moyenne/médiane par zone géographique",
                "Création d'indicateurs de données manquantes",
                "Agrégation en variables de synthèse (ex: moyenne par saison)",
                "Utilisation des stations météo les plus proches"
            ]
        },
        'Building': {
            'description': "Caractéristiques physiques des bâtiments",
            'options': [
                "Imputation par mode pour les catégories similaires",
                "Création de catégories 'UNKNOWN'",
                "Utilisation des corrélations entre surface/hauteur/nombre de bâtiments",
                "Agrégation des caractéristiques similaires"
            ]
        },
        'Insurance': {
            'description': "Variables d'assurance et risque",
            'options': [
                "Conservation stricte - pas d'imputation pour les variables critiques",
                "Imputation basée sur les profils de risque similaires",
                "Utilisation des règles métier pour l'imputation",
                "Création de catégories de risque agrégées"
            ]
        },
        'Geographic': {
            'description': "Variables de localisation et environnement",
            'options': [
                "Imputation par plus proche voisin géographique",
                "Agrégation à un niveau géographique plus large",
                "Création de clusters géographiques",
                "Utilisation des données d'occupation des sols"
            ]
        },
        'Demographic': {
            'description': "Variables socio-démographiques",
            'options': [
                "Imputation par la moyenne du quartier/zone",
                "Utilisation des données INSEE du niveau supérieur",
                "Création d'indices composites",
                "Agrégation par profil démographique"
            ]
        },
        'Activity': {
            'description': "Variables d'activité et équipement",
            'options': [
                "Imputation basée sur l'activité principale",
                "Création de groupes d'activité simplifiés",
                "Utilisation des corrélations activité/équipement",
                "Conservation uniquement des équipements principaux"
            ]
        },
        'Target': {
            'description': "Variables cibles (FREQ, CM, CHARGE)",
            'options': [
                "Aucune imputation - suppression des lignes",
                "Analyse séparée pour chaque variable cible",
                "Création d'indicateurs de fiabilité",
                "Modélisation spécifique pour les valeurs manquantes"
            ]
        },
        'ID': {
            'description': "Variables d'identification",
            'options': [
                "Aucune imputation - suppression des lignes",
                "Création de nouveaux identifiants",
                "Vérification des doublons et incohérences",
                "Traçabilité des modifications"
            ]
        }
    }
    
    # Affichage des stratégies
    for group, info in strategies.items():
        print(f"\n{group} - {info['description']}:")
        print("Stratégies proposées:")
        for i, opt in enumerate(info['options'], 1):
            print(f"{i}. {opt}")
        
        # Afficher les variables concernées
        if group == 'Weather':
            vars_list = sum(WEATHER_VARS.values(), [])
        elif group == 'Building':
            vars_list = sum(BUILDING_VARS.values(), [])
        elif group == 'Insurance':
            vars_list = sum([v if isinstance(v, list) else sum(v.values(), []) 
                           for v in INSURANCE_VARS.values()], [])
        elif group == 'Geographic':
            vars_list = sum([v if isinstance(v, list) else sum(v.values(), []) 
                           for v in GEOGRAPHIC_VARS.values()], [])
        elif group == 'Demographic':
            vars_list = sum([sum(v.values(), []) for v in DEMOGRAPHIC_VARS.values()], [])
        elif group == 'Activity':
            vars_list = sum(ACTIVITY_VARS.values(), [])
        elif group == 'Target':
            vars_list = sum([v for v in TARGET_VARS['primary'].values()], [])
        elif group == 'ID':
            vars_list = ID_VARS['identifiers']
            
        print(f"\nNombre de variables: {len(vars_list)}")
        print(f"Exemples: {vars_list[:3]}...")
        print("-" * 80)
    
    return strategies

# Exécuter l'analyse
strategies = propose_strategies()

In [ ]:
def aggregate_weather_vars(train_df, test_df):
    print("=== Traitement des variables météorologiques ===")
    
    # Utiliser la structure existante de WEATHER_VARS
    weather_groups = {
        'temp_max': [col for col in train_df.columns if any(x in col for x in ['NBJTX', 'TX', 'TMMAX', 'TXMAX'])],
        'temp_min': [col for col in train_df.columns if any(x in col for x in ['NBJTN', 'TN', 'TMMIN', 'TNMIN'])],
        'temp_moy': [col for col in train_df.columns if any(x in col for x in ['TM', 'TMM'])],
        'temp_amplitude': [col for col in train_df.columns if 'TAMPLI' in col],
        'wind': [col for col in train_df.columns if any(x in col for x in [
            'NBJFF', 'NBJFXI3S', 'NBJFXY', 'FFM_VOR', 'FXI3SAB_VOR', 
            'FXIAB_VOR', 'FXYAB_VOR', 'FFM_vor_com', 'FXI3SAB_vor_com'
        ])],
        'rain': [col for col in train_df.columns if any(x in col for x in [
            'NBJRR', 'RR_VOR', 'RRAB_VOR'
        ])]
    }
    
    # Types de mesures
    measure_types = {
        'moyenne': 'MM_A',    # Moyenne annuelle
        'maximum': 'MMAX_A',  # Maximum annuel
        'somme': 'MSOM_A'    # Somme annuelle
    }
    
    new_weather_cols = []
    
    # Pour chaque groupe de variables météo
    for weather_type, cols in weather_groups.items():
        print(f"\n{weather_type.capitalize()} - {len(cols)} variables")
        print("Exemples de colonnes:", cols[:3])
        
        # Pour chaque type de mesure
        for measure_name, measure_suffix in measure_types.items():
            measure_cols = [col for col in cols if measure_suffix in col]
            
            if measure_cols:
                # Créer différentes agrégations
                aggregations = {
                    'mean': f'{weather_type}_{measure_name}_mean',
                    'min': f'{weather_type}_{measure_name}_min',
                    'max': f'{weather_type}_{measure_name}_max'
                }
                
                for agg_type, col_name in aggregations.items():
                    if agg_type == 'mean':
                        train_df[col_name] = train_df[measure_cols].mean(axis=1)
                        test_df[col_name] = test_df[measure_cols].mean(axis=1)
                    elif agg_type == 'min':
                        train_df[col_name] = train_df[measure_cols].min(axis=1)
                        test_df[col_name] = test_df[measure_cols].min(axis=1)
                    elif agg_type == 'max':
                        train_df[col_name] = train_df[measure_cols].max(axis=1)
                        test_df[col_name] = test_df[measure_cols].max(axis=1)
                    
                    new_weather_cols.append(col_name)
    
    # Créer des ratios et variations
    for weather_type in ['temp_max', 'temp_min', 'wind', 'rain']:
        mean_cols = [col for col in new_weather_cols if weather_type in col and 'mean' in col]
        max_cols = [col for col in new_weather_cols if weather_type in col and 'max' in col]
        
        if mean_cols and max_cols:
            # Ratio max/mean
            ratio_col = f'{weather_type}_max_mean_ratio'
            train_df[ratio_col] = train_df[max_cols[0]] / train_df[mean_cols[0]]
            test_df[ratio_col] = test_df[max_cols[0]] / test_df[mean_cols[0]]
            new_weather_cols.append(ratio_col)
    
    # Identifier les colonnes à supprimer
    weather_cols_to_drop = sum(weather_groups.values(), [])
    
    # Statistiques des nouvelles variables
    print("\n=== Statistiques des nouvelles variables météo ===")
    print("\nNombre de variables créées:", len(new_weather_cols))
    print("\nExemples de nouvelles variables:", new_weather_cols[:5])
    
    print("\nStatistiques descriptives:")
    print(train_df[new_weather_cols].describe())
    
    print("\nValeurs manquantes dans les nouvelles variables:")
    missing_pct = (train_df[new_weather_cols].isnull().mean() * 100).round(2)
    print(missing_pct[missing_pct > 0])
    
    return {
        'to_drop': weather_cols_to_drop,
        'new_cols': new_weather_cols,
        'stats': {
            'n_original': len(weather_cols_to_drop),
            'n_new': len(new_weather_cols),
            'missing_pct': missing_pct.to_dict()
        }
    }

# Exécuter la fonction
weather_results = handle_weather_vars(train_df, test_df)

# Afficher un résumé détaillé
print("\n=== Résumé des modifications ===")
print(f"Variables originales: {weather_results['stats']['n_original']}")
print(f"Nouvelles variables: {weather_results['stats']['n_new']}")
print("\nRatio de réduction:", 
      round(weather_results['stats']['n_new'] / weather_results['stats']['n_original'], 2))

In [ ]:
def handle_building_vars(train_df, test_df):
    print("=== Traitement des variables bâtiment ===")
    
    new_building_cols = []
    
    # 1. Consolider les hauteurs
    height_cols = BUILDING_VARS['height']
    if height_cols:
        print("\nConsolidation des hauteurs:")
        height_aggs = {
            'height_mean': 'mean',
            'height_max': 'max',
            'height_min': 'min'
        }
        
        for col_name, agg_func in height_aggs.items():
            train_df[col_name] = train_df[height_cols].agg(agg_func, axis=1)
            test_df[col_name] = test_df[height_cols].agg(agg_func, axis=1)
            new_building_cols.append(col_name)
        
        print(f"Créé {len(height_aggs)} variables de hauteur agrégées")
    
    # 2. Consolider les surfaces
    surface_cols = BUILDING_VARS['surface']
    if surface_cols:
        print("\nConsolidation des surfaces:")
        surface_aggs = {
            'surface_total': 'sum',
            'surface_mean': 'mean',
            'surface_max': 'max',
            'n_surfaces': lambda x: x.notna().sum()  # Nombre de surfaces renseignées
        }
        
        for col_name, agg_func in surface_aggs.items():
            train_df[col_name] = train_df[surface_cols].agg(agg_func, axis=1)
            test_df[col_name] = test_df[surface_cols].agg(agg_func, axis=1)
            new_building_cols.append(col_name)
        
        print(f"Créé {len(surface_aggs)} variables de surface agrégées")
    
    # 3. Consolider le nombre de bâtiments
    building_cols = BUILDING_VARS['buildings']
    if building_cols:
        print("\nConsolidation du nombre de bâtiments:")
        building_aggs = {
            'buildings_total': 'sum',
            'buildings_count': lambda x: x.notna().sum(),  # Nombre de types de bâtiments
            'buildings_types': lambda x: (x > 0).sum()     # Nombre de types avec bâtiments
        }
        
        for col_name, agg_func in building_aggs.items():
            train_df[col_name] = train_df[building_cols].agg(agg_func, axis=1)
            test_df[col_name] = test_df[building_cols].agg(agg_func, axis=1)
            new_building_cols.append(col_name)
        
        print(f"Créé {len(building_aggs)} variables de comptage de bâtiments")
    
    # 4. Créer des ratios et indicateurs composites
    print("\nCréation d'indicateurs composites:")
    
    # Ratio surface/bâtiment
    if 'surface_total' in new_building_cols and 'buildings_total' in new_building_cols:
        train_df['surface_per_building'] = (train_df['surface_total'] / 
                                          train_df['buildings_total'].replace(0, np.nan))
        test_df['surface_per_building'] = (test_df['surface_total'] / 
                                         test_df['buildings_total'].replace(0, np.nan))
        new_building_cols.append('surface_per_building')
    
    # Ratio hauteur/surface
    if 'height_mean' in new_building_cols and 'surface_total' in new_building_cols:
        train_df['height_surface_ratio'] = (train_df['height_mean'] / 
                                          train_df['surface_total'].replace(0, np.nan))
        test_df['height_surface_ratio'] = (test_df['height_mean'] / 
                                         test_df['surface_total'].replace(0, np.nan))
        new_building_cols.append('height_surface_ratio')
    
    # 5. Encoder les caractéristiques catégorielles
    char_cols = BUILDING_VARS['characteristics']
    if char_cols:
        print("\nEncodage des caractéristiques:")
        for col in char_cols:
            dummies = pd.get_dummies(train_df[col], prefix=f'char_{col}')
            test_dummies = pd.get_dummies(test_df[col], prefix=f'char_{col}')
            
            # Assurer les mêmes colonnes dans train et test
            for dummy_col in dummies.columns:
                if dummy_col not in test_dummies.columns:
                    test_dummies[dummy_col] = 0
            
            train_df = pd.concat([train_df, dummies], axis=1)
            test_df = pd.concat([test_df, test_dummies], axis=1)
            new_building_cols.extend(dummies.columns)
        
        print(f"Créé {len(dummies.columns)} variables dummy pour les caractéristiques")
    
    # Identifier les colonnes à supprimer
    building_cols_to_drop = sum([
        BUILDING_VARS['height'],
        BUILDING_VARS['surface'],
        BUILDING_VARS['buildings'],
        BUILDING_VARS['characteristics']
    ], [])
    
    # Statistiques des nouvelles variables
    print("\n=== Statistiques des nouvelles variables bâtiment ===")
    print(f"\nNombre de variables créées: {len(new_building_cols)}")
    print(f"Nombre de variables à supprimer: {len(building_cols_to_drop)}")
    
    print("\nStatistiques descriptives des variables numériques:")
    numeric_cols = [col for col in new_building_cols 
                   if train_df[col].dtype in ['int64', 'float64']]
    print(train_df[numeric_cols].describe())
    
    return {
        'to_drop': building_cols_to_drop,
        'new_cols': new_building_cols,
        'train_df': train_df,
        'test_df': test_df
    }

# Exécuter la fonction
building_results = handle_building_vars(train_df, test_df)

# Mettre à jour les DataFrames
train_df = building_results['train_df']
test_df = building_results['test_df']

In [ ]:
def handle_geographic_vars(train_df, test_df):
    print("=== Traitement des variables géographiques ===")
    
    new_geo_cols = []
    
    # 1. Traiter les distances
    if 'distances' in GEOGRAPHIC_VARS:
        print("\nTraitement des distances:")
        
        # Pour chaque type d'environnement
        for env_type, cols in GEOGRAPHIC_VARS['distances'].items():
            if cols:
                print(f"\n{env_type.capitalize()} - {len(cols)} variables")
                
                # Créer différentes agrégations
                aggs = {
                    f'distance_{env_type}_mean': 'mean',
                    f'distance_{env_type}_min': 'min',  # Distance la plus proche
                    f'distance_{env_type}_max': 'max'   # Distance la plus éloignée
                }
                
                for col_name, agg_func in aggs.items():
                    train_df[col_name] = train_df[cols].agg(agg_func, axis=1)
                    test_df[col_name] = test_df[cols].agg(agg_func, axis=1)
                    new_geo_cols.append(col_name)
                
                # Nombre d'éléments proches (distance < seuil)
                thresholds = [1000, 5000]  # 1km, 5km
                for threshold in thresholds:
                    col_name = f'n_{env_type}_within_{threshold}m'
                    train_df[col_name] = (train_df[cols] < threshold).sum(axis=1)
                    test_df[col_name] = (test_df[cols] < threshold).sum(axis=1)
                    new_geo_cols.append(col_name)
    
    # 2. Traiter les proportions
    if 'proportions' in GEOGRAPHIC_VARS:
        print("\nTraitement des proportions:")
        
        for env_type, cols in GEOGRAPHIC_VARS['proportions'].items():
            if cols:
                print(f"\n{env_type.capitalize()} - {len(cols)} variables")
                
                # Somme des proportions par type
                col_name = f'proportion_{env_type}_total'
                train_df[col_name] = train_df[cols].sum(axis=1)
                test_df[col_name] = test_df[cols].sum(axis=1)
                new_geo_cols.append(col_name)
                
                # Proportion dominante
                col_name = f'proportion_{env_type}_max'
                train_df[col_name] = train_df[cols].max(axis=1)
                test_df[col_name] = test_df[cols].max(axis=1)
                new_geo_cols.append(col_name)
                
                # Diversité (nombre de types > seuil)
                col_name = f'diversity_{env_type}'
                threshold = 0.1  # 10%
                train_df[col_name] = (train_df[cols] > threshold).sum(axis=1)
                test_df[col_name] = (test_df[cols] > threshold).sum(axis=1)
                new_geo_cols.append(col_name)
    
    # 3. Traiter la topographie
    if 'topography' in GEOGRAPHIC_VARS:
        print("\nTraitement de la topographie:")
        
        # Altitude
        altitude_cols = GEOGRAPHIC_VARS['topography']['altitude']
        if altitude_cols:
            altitude_aggs = {
                'altitude_mean': 'mean',
                'altitude_min': 'min',
                'altitude_max': 'max',
                'altitude_range': lambda x: x.max() - x.min()
            }
            
            for col_name, agg_func in altitude_aggs.items():
                train_df[col_name] = train_df[altitude_cols].agg(agg_func, axis=1)
                test_df[col_name] = test_df[altitude_cols].agg(agg_func, axis=1)
                new_geo_cols.append(col_name)
    
    # 4. Traiter les services d'urgence
    if 'emergency' in GEOGRAPHIC_VARS:
        print("\nTraitement des services d'urgence:")
        emergency_cols = GEOGRAPHIC_VARS['emergency']
        # Garder ces variables telles quelles car déjà agrégées
        new_geo_cols.extend(emergency_cols)
    
    # 5. Créer des indicateurs composites
    print("\nCréation d'indicateurs composites:")
    
    # Ratio urbain/naturel
    if 'proportion_urban_total' in new_geo_cols and 'proportion_natural_total' in new_geo_cols:
        col_name = 'urban_natural_ratio'
        train_df[col_name] = (train_df['proportion_urban_total'] / 
                             train_df['proportion_natural_total'].replace(0, np.nan))
        test_df[col_name] = (test_df['proportion_urban_total'] / 
                            test_df['proportion_natural_total'].replace(0, np.nan))
        new_geo_cols.append(col_name)
    
    # Identifier les colonnes à supprimer
    geo_cols_to_drop = sum([
        sum(GEOGRAPHIC_VARS['distances'].values(), []),
        sum(GEOGRAPHIC_VARS['proportions'].values(), []),
        GEOGRAPHIC_VARS['topography']['altitude']
    ], [])
    
    # Statistiques des nouvelles variables
    print("\n=== Statistiques des nouvelles variables géographiques ===")
    print(f"\nNombre de variables créées: {len(new_geo_cols)}")
    print(f"Nombre de variables à supprimer: {len(geo_cols_to_drop)}")
    
    print("\nStatistiques descriptives:")
    print(train_df[new_geo_cols].describe())
    
    return {
        'to_drop': geo_cols_to_drop,
        'new_cols': new_geo_cols,
        'train_df': train_df,
        'test_df': test_df
    }

# Exécuter la fonction
geo_results = handle_geographic_vars(train_df, test_df)

# Mettre à jour les DataFrames
train_df = geo_results['train_df']
test_df = geo_results['test_df']

In [ ]:
def handle_insurance_vars(train_df, test_df):
    print("=== Traitement des variables d'assurance ===")
    
    new_insurance_cols = []
    
    # 1. Traitement des indicateurs de risque
    if 'risk_indicators' in INSURANCE_VARS:
        print("\nTraitement des indicateurs de risque:")
        risk_cols = INSURANCE_VARS['risk_indicators']
        
        # Agrégations des indicateurs de risque
        risk_aggs = {
            'risk_mean': 'mean',
            'risk_max': 'max',
            'risk_sum': 'sum',
            'n_risks': lambda x: (x > 0).sum()  # Nombre de risques actifs
        }
        
        for col_name, agg_func in risk_aggs.items():
            train_df[col_name] = train_df[risk_cols].agg(agg_func, axis=1)
            test_df[col_name] = test_df[risk_cols].agg(agg_func, axis=1)
            new_insurance_cols.append(col_name)
    
    # 2. Traitement des capitaux
    if 'capital' in INSURANCE_VARS:
        print("\nTraitement des capitaux:")
        capital_cols = INSURANCE_VARS['capital']
        
        # Agrégations des capitaux
        capital_aggs = {
            'capital_total': 'sum',
            'capital_mean': 'mean',
            'capital_max': 'max',
            'n_capitals': lambda x: x.notna().sum()  # Nombre de capitaux renseignés
        }
        
        for col_name, agg_func in capital_aggs.items():
            train_df[col_name] = train_df[capital_cols].agg(agg_func, axis=1)
            test_df[col_name] = test_df[capital_cols].agg(agg_func, axis=1)
            new_insurance_cols.append(col_name)
    
    # 3. Traitement des dérogations
    if 'derogation' in INSURANCE_VARS:
        print("\nTraitement des dérogations:")
        derog_cols = INSURANCE_VARS['derogation']
        
        # Nombre total de dérogations
        train_df['n_derogations'] = (train_df[derog_cols] > 0).sum(axis=1)
        test_df['n_derogations'] = (test_df[derog_cols] > 0).sum(axis=1)
        new_insurance_cols.append('n_derogations')
        
        # Encodage one-hot des dérogations
        for col in derog_cols:
            dummies = pd.get_dummies(train_df[col], prefix=f'derog_{col}')
            test_dummies = pd.get_dummies(test_df[col], prefix=f'derog_{col}')
            
            # Assurer les mêmes colonnes dans train et test
            for dummy_col in dummies.columns:
                if dummy_col not in test_dummies.columns:
                    test_dummies[dummy_col] = 0
            
            train_df = pd.concat([train_df, dummies], axis=1)
            test_df = pd.concat([test_df, test_dummies], axis=1)
            new_insurance_cols.extend(dummies.columns)
    
    # 4. Traitement des termes du contrat
    if 'contract_terms' in INSURANCE_VARS:
        print("\nTraitement des termes du contrat:")
        
        # Encoder les franchises et indemnisations
        for term_type, cols in INSURANCE_VARS['contract_terms'].items():
            for col in cols:
                dummies = pd.get_dummies(train_df[col], prefix=f'{term_type}_{col}')
                test_dummies = pd.get_dummies(test_df[col], prefix=f'{term_type}_{col}')
                
                # Assurer les mêmes colonnes
                for dummy_col in dummies.columns:
                    if dummy_col not in test_dummies.columns:
                        test_dummies[dummy_col] = 0
                
                train_df = pd.concat([train_df, dummies], axis=1)
                test_df = pd.concat([test_df, test_dummies], axis=1)
                new_insurance_cols.extend(dummies.columns)
    
    # 5. Traitement de l'historique
    if 'history' in INSURANCE_VARS:
        print("\nTraitement de l'historique:")
        history_cols = INSURANCE_VARS['history']
        
        # Garder ces variables telles quelles
        new_insurance_cols.extend(history_cols)
        
        # Créer des ratios
        if 'NBSINCONJ' in history_cols and 'NBSINSTRT' in history_cols:
            train_df['ratio_sinistres'] = (train_df['NBSINCONJ'] / 
                                         train_df['NBSINSTRT'].replace(0, np.nan))
            test_df['ratio_sinistres'] = (test_df['NBSINCONJ'] / 
                                        test_df['NBSINSTRT'].replace(0, np.nan))
            new_insurance_cols.append('ratio_sinistres')
    
    # Identifier les colonnes à supprimer
    insurance_cols_to_drop = sum([
        INSURANCE_VARS['risk_indicators'],
        INSURANCE_VARS['capital'],
        INSURANCE_VARS['derogation'],
        sum(INSURANCE_VARS['contract_terms'].values(), [])
    ], [])
    
    # Statistiques des nouvelles variables
    print("\n=== Statistiques des nouvelles variables d'assurance ===")
    print(f"\nNombre de variables créées: {len(new_insurance_cols)}")
    print(f"Nombre de variables à supprimer: {len(insurance_cols_to_drop)}")
    
    print("\nStatistiques descriptives des variables numériques:")
    numeric_cols = [col for col in new_insurance_cols 
                   if train_df[col].dtype in ['int64', 'float64']]
    print(train_df[numeric_cols].describe())
    
    return {
        'to_drop': insurance_cols_to_drop,
        'new_cols': new_insurance_cols,
        'train_df': train_df,
        'test_df': test_df
    }

# Exécuter la fonction
insurance_results = handle_insurance_vars(train_df, test_df)

# Mettre à jour les DataFrames
train_df = insurance_results['train_df']
test_df = insurance_results['test_df']

In [ ]:
def handle_demographic_vars(train_df, test_df):
    print("=== Traitement des variables démographiques ===")
    
    new_demo_cols = []
    
    # 1. Traitement des ménages
    if 'household' in DEMOGRAPHIC_VARS:
        print("\nTraitement des ménages:")
        
        # Variables de base des ménages
        men_total = DEMOGRAPHIC_VARS['household']['general'][0]  # 'MEN'
        
        # Ratios pour la composition des ménages
        if 'composition' in DEMOGRAPHIC_VARS['household']:
            comp_cols = DEMOGRAPHIC_VARS['household']['composition']
            for col in comp_cols:
                ratio_name = f'ratio_{col}'
                train_df[ratio_name] = train_df[col] / train_df[men_total]
                test_df[ratio_name] = test_df[col] / test_df[men_total]
                new_demo_cols.append(ratio_name)
        
        # Ratios pour le type de logement
        if 'housing_type' in DEMOGRAPHIC_VARS['household']:
            house_cols = DEMOGRAPHIC_VARS['household']['housing_type']
            for col in house_cols:
                ratio_name = f'ratio_{col}'
                train_df[ratio_name] = train_df[col] / train_df[men_total]
                test_df[ratio_name] = test_df[col] / test_df[men_total]
                new_demo_cols.append(ratio_name)
        
        # Indicateurs économiques
        if 'economic' in DEMOGRAPHIC_VARS['household']:
            eco_cols = DEMOGRAPHIC_VARS['household']['economic']
            
            # Ratio de ménages pauvres
            if 'men_pauv' in eco_cols:
                train_df['ratio_pauvrete'] = train_df['men_pauv'] / train_df[men_total]
                test_df['ratio_pauvrete'] = test_df['men_pauv'] / test_df[men_total]
                new_demo_cols.append('ratio_pauvrete')
            
            # Surface moyenne par ménage
            if 'men_surf' in eco_cols:
                train_df['surface_per_menage'] = train_df['men_surf'] / train_df[men_total]
                test_df['surface_per_menage'] = test_df['men_surf'] / test_df[men_total]
                new_demo_cols.append('surface_per_menage')
    
    # 2. Traitement des logements
    if 'housing' in DEMOGRAPHIC_VARS:
        print("\nTraitement des logements:")
        
        # Périodes de construction
        if 'construction_period' in DEMOGRAPHIC_VARS['housing']:
            period_cols = DEMOGRAPHIC_VARS['housing']['construction_period']
            total_log = train_df[period_cols].sum(axis=1)
            
            # Ratios par période
            for col in period_cols:
                ratio_name = f'ratio_{col}'
                train_df[ratio_name] = train_df[col] / total_log
                test_df[ratio_name] = test_df[col] / total_log
                new_demo_cols.append(ratio_name)
            
            # Âge moyen du parc (pondéré)
            # Supposons: avA1 = avant 1950, A1_A2 = 1950-1975, A2_A3 = 1975-2000, apA3 = après 2000
            weights = {'log_avA1': 1940, 'log_A1_A2': 1962.5, 'log_A2_A3': 1987.5, 'log_apA3': 2010}
            weighted_sum = sum(train_df[col] * weight for col, weight in weights.items())
            train_df['age_moyen_logements'] = weighted_sum / total_log
            test_df['age_moyen_logements'] = sum(test_df[col] * weight for col, weight in weights.items()) / total_log
            new_demo_cols.append('age_moyen_logements')
    
    # 3. Traitement des individus
    if 'individual' in DEMOGRAPHIC_VARS:
        print("\nTraitement des individus:")
        
        # Total des individus
        ind_total = DEMOGRAPHIC_VARS['individual']['total'][0]  # 'IND'
        
        # Traitement des groupes d'âge
        if 'age_groups' in DEMOGRAPHIC_VARS['individual']:
            age_cols = DEMOGRAPHIC_VARS['individual']['age_groups']
            
            # Ratios par groupe d'âge
            for col in age_cols:
                ratio_name = f'ratio_{col}'
                train_df[ratio_name] = train_df[col] / train_df[ind_total]
                test_df[ratio_name] = test_df[col] / test_df[ind_total]
                new_demo_cols.append(ratio_name)
            
            # Indices démographiques
            # Indice de jeunesse (0-Y4 / Y5-Y9)
            young_cols = [col for col in age_cols if any(f'_{i}_' in col for i in range(5))]
            old_cols = [col for col in age_cols if any(f'_{i}_' in col for i in range(5, 10))]
            
            train_df['indice_jeunesse'] = train_df[young_cols].sum(axis=1) / train_df[old_cols].sum(axis=1)
            test_df['indice_jeunesse'] = test_df[young_cols].sum(axis=1) / test_df[old_cols].sum(axis=1)
            new_demo_cols.append('indice_jeunesse')
        
        # Autres indicateurs
        if 'other' in DEMOGRAPHIC_VARS['individual']:
            other_cols = DEMOGRAPHIC_VARS['individual']['other']
            
            # Niveau de vie moyen
            if 'ind_snv' in other_cols:
                train_df['niveau_vie_moyen'] = train_df['ind_snv'] / train_df[ind_total]
                test_df['niveau_vie_moyen'] = test_df['ind_snv'] / test_df[ind_total]
                new_demo_cols.append('niveau_vie_moyen')
    
    # 4. Indicateurs composites
    print("\nCréation d'indicateurs composites:")
    
    # Densité de population
    if 'MEN' in train_df.columns and 'men_surf' in train_df.columns:
        train_df['densite_population'] = train_df['IND'] / train_df['men_surf']
        test_df['densite_population'] = test_df['IND'] / test_df['men_surf']
        new_demo_cols.append('densite_population')
    
    # Taille moyenne des ménages
    train_df['taille_moyenne_menage'] = train_df['IND'] / train_df['MEN']
    test_df['taille_moyenne_menage'] = test_df['IND'] / test_df['MEN']
    new_demo_cols.append('taille_moyenne_menage')
    
    # Identifier les colonnes à supprimer
    demo_cols_to_drop = sum([
        sum([cols for cols in DEMOGRAPHIC_VARS['household'].values()], []),
        sum([cols for cols in DEMOGRAPHIC_VARS['housing'].values()], []),
        sum([cols for cols in DEMOGRAPHIC_VARS['individual'].values()], [])
    ], [])
    
    # Statistiques des nouvelles variables
    print("\n=== Statistiques des nouvelles variables démographiques ===")
    print(f"\nNombre de variables créées: {len(new_demo_cols)}")
    print(f"Nombre de variables à supprimer: {len(demo_cols_to_drop)}")
    
    print("\nStatistiques descriptives:")
    print(train_df[new_demo_cols].describe())
    
    return {
        'to_drop': demo_cols_to_drop,
        'new_cols': new_demo_cols,
        'train_df': train_df,
        'test_df': test_df
    }

# Exécuter la fonction
demographic_results = handle_demographic_vars(train_df, test_df)

# Mettre à jour les DataFrames
train_df = demographic_results['train_df']
test_df = demographic_results['test_df']

In [ ]:
def handle_activity_vars(train_df, test_df):
    print("=== Traitement des variables d'activité ===")
    
    new_activity_cols = []
    
    # 1. Traitement des équipements
    if 'equipment' in ACTIVITY_VARS:
        print("\nTraitement des équipements:")
        equip_cols = ACTIVITY_VARS['equipment']
        
        # Nombre total d'équipements
        train_df['n_equipements_total'] = (train_df[equip_cols] > 0).sum(axis=1)
        test_df['n_equipements_total'] = (test_df[equip_cols] > 0).sum(axis=1)
        new_activity_cols.append('n_equipements_total')
        
        # Score de complexité des équipements (pondéré par le numéro d'équipement)
        equip_weights = {col: int(col.replace('EQUIPEMENT', '')) for col in equip_cols}
        train_df['score_complexite_equip'] = sum(train_df[col] * weight 
                                               for col, weight in equip_weights.items())
        test_df['score_complexite_equip'] = sum(test_df[col] * weight 
                                              for col, weight in equip_weights.items())
        new_activity_cols.append('score_complexite_equip')
        
        # Encodage one-hot des équipements
        for col in equip_cols:
            dummies = pd.get_dummies(train_df[col], prefix=f'equip_{col}')
            test_dummies = pd.get_dummies(test_df[col], prefix=f'equip_{col}')
            
            # Assurer les mêmes colonnes
            for dummy_col in dummies.columns:
                if dummy_col not in test_dummies.columns:
                    test_dummies[dummy_col] = 0
            
            train_df = pd.concat([train_df, dummies], axis=1)
            test_df = pd.concat([test_df, test_dummies], axis=1)
            new_activity_cols.extend(dummies.columns)
    
    # 2. Traitement de l'activité et vocation
    if 'activity' in ACTIVITY_VARS:
        print("\nTraitement de l'activité et vocation:")
        activity_cols = ACTIVITY_VARS['activity']
        
        # Encodage des variables catégorielles
        for col in activity_cols:
            # Encodage avec gestion de la fréquence
            encoding_map = (train_df[col].value_counts() / len(train_df)).to_dict()
            
            # Appliquer l'encodage
            train_df[f'{col}_freq'] = train_df[col].map(encoding_map)
            test_df[f'{col}_freq'] = test_df[col].map(encoding_map)
            new_activity_cols.append(f'{col}_freq')
            
            # Encodage one-hot classique
            dummies = pd.get_dummies(train_df[col], prefix=col)
            test_dummies = pd.get_dummies(test_df[col], prefix=col)
            
            # Assurer les mêmes colonnes
            for dummy_col in dummies.columns:
                if dummy_col not in test_dummies.columns:
                    test_dummies[dummy_col] = 0
            
            train_df = pd.concat([train_df, dummies], axis=1)
            test_df = pd.concat([test_df, test_dummies], axis=1)
            new_activity_cols.extend(dummies.columns)
        
        # Créer des combinaisons d'activité-vocation si disponibles
        if 'ACTIVIT2' in activity_cols and 'VOCATION' in activity_cols:
            train_df['activ_voc'] = train_df['ACTIVIT2'] + '_' + train_df['VOCATION']
            test_df['activ_voc'] = test_df['ACTIVIT2'] + '_' + test_df['VOCATION']
            
            # Encoder cette nouvelle combinaison
            dummies = pd.get_dummies(train_df['activ_voc'], prefix='activ_voc')
            test_dummies = pd.get_dummies(test_df['activ_voc'], prefix='activ_voc')
            
            # Assurer les mêmes colonnes
            for dummy_col in dummies.columns:
                if dummy_col not in test_dummies.columns:
                    test_dummies[dummy_col] = 0
            
            train_df = pd.concat([train_df, dummies], axis=1)
            test_df = pd.concat([test_df, test_dummies], axis=1)
            new_activity_cols.extend(dummies.columns)
    
    # 3. Traitement de la taille et catégorie
    if 'size_category' in ACTIVITY_VARS:
        print("\nTraitement de la taille et catégorie:")
        size_cols = [col for col in ACTIVITY_VARS['size_category'] if col.startswith('TAILLE')]
        
        # Score de taille global
        if size_cols:
            train_df['score_taille'] = train_df[size_cols].mean(axis=1)
            test_df['score_taille'] = test_df[size_cols].mean(axis=1)
            new_activity_cols.append('score_taille')
            
            # Variabilité de la taille
            train_df['var_taille'] = train_df[size_cols].std(axis=1)
            test_df['var_taille'] = test_df[size_cols].std(axis=1)
            new_activity_cols.append('var_taille')
        
        # Traitement de COEFASS
        if 'COEFASS' in ACTIVITY_VARS['size_category']:
            # Encodage fréquentiel
            coef_map = (train_df['COEFASS'].value_counts() / len(train_df)).to_dict()
            train_df['coefass_freq'] = train_df['COEFASS'].map(coef_map)
            test_df['coefass_freq'] = test_df['COEFASS'].map(coef_map)
            new_activity_cols.append('coefass_freq')
            
            # Encodage one-hot
            dummies = pd.get_dummies(train_df['COEFASS'], prefix='coefass')
            test_dummies = pd.get_dummies(test_df['COEFASS'], prefix='coefass')
            
            # Assurer les mêmes colonnes
            for dummy_col in dummies.columns:
                if dummy_col not in test_dummies.columns:
                    test_dummies[dummy_col] = 0
            
            train_df = pd.concat([train_df, dummies], axis=1)
            test_df = pd.concat([test_df, test_dummies], axis=1)
            new_activity_cols.extend(dummies.columns)
    
    # 4. Traitement des services d'urgence
    if 'emergency' in ACTIVITY_VARS:
        print("\nTraitement des services d'urgence:")
        emergency_cols = ACTIVITY_VARS['emergency']
        # Garder ces variables telles quelles
        new_activity_cols.extend(emergency_cols)
    
    # Identifier les colonnes à supprimer
    activity_cols_to_drop = sum([
        ACTIVITY_VARS['equipment'],
        ACTIVITY_VARS['activity'],
        ACTIVITY_VARS['size_category']
    ], [])
    
    # Statistiques des nouvelles variables
    print("\n=== Statistiques des nouvelles variables d'activité ===")
    print(f"\nNombre de variables créées: {len(new_activity_cols)}")
    print(f"Nombre de variables à supprimer: {len(activity_cols_to_drop)}")
    
    print("\nStatistiques descriptives des variables numériques:")
    numeric_cols = [col for col in new_activity_cols 
                   if train_df[col].dtype in ['int64', 'float64']]
    print(train_df[numeric_cols].describe())
    
    return {
        'to_drop': activity_cols_to_drop,
        'new_cols': new_activity_cols,
        'train_df': train_df,
        'test_df': test_df
    }

# Exécuter la fonction
activity_results = handle_activity_vars(train_df, test_df)

# Mettre à jour les DataFrames
train_df = activity_results['train_df']
test_df = activity_results['test_df']

In [ ]:
def handle_target_vars(train_df, test_df):
    print("=== Traitement des variables cibles ===")
    
    new_target_cols = []
    target_stats = {}
    
    # Récupérer les variables cibles
    freq_var = TARGET_VARS['primary']['frequency'][0]  # 'FREQ'
    cost_var = TARGET_VARS['primary']['cost'][0]       # 'CM'
    charge_var = TARGET_VARS['primary']['total'][0]    # 'CHARGE'
    
    # 1. Vérification de la cohérence des calculs
    print("\nVérification de la cohérence des calculs:")
    
    if all(var in train_df.columns for var in [freq_var, cost_var, charge_var, 'ANNEE_ASSURANCE']):
        # Calculer CHARGE théorique
        train_df['CHARGE_calc'] = (train_df[freq_var] * 
                                  train_df[cost_var] * 
                                  train_df['ANNEE_ASSURANCE'])
        
        # Calculer la différence relative
        train_df['charge_diff_pct'] = abs(train_df['CHARGE_calc'] - 
                                        train_df[charge_var]) / train_df[charge_var] * 100
        
        # Statistiques sur les différences
        diff_stats = train_df['charge_diff_pct'].describe()
        print("\nDifférences relatives dans le calcul de CHARGE:")
        print(diff_stats)
        
        # Marquer les incohérences importantes
        threshold = diff_stats['75%'] + 1.5 * (diff_stats['75%'] - diff_stats['25%'])  # IQR method
        train_df['charge_incoherent'] = train_df['charge_diff_pct'] > threshold
        new_target_cols.extend(['charge_diff_pct', 'charge_incoherent'])
        
        target_stats['charge_coherence'] = {
            'mean_diff_pct': diff_stats['mean'],
            'median_diff_pct': diff_stats['50%'],
            'n_incoherent': train_df['charge_incoherent'].sum()
        }
    
    # 2. Traitement des valeurs extrêmes
    print("\nTraitement des valeurs extrêmes:")
    
    for var, var_type in zip([freq_var, cost_var, charge_var], ['freq', 'cost', 'charge']):
        if var in train_df.columns:
            # Calculer les quantiles
            q1 = train_df[var].quantile(0.25)
            q3 = train_df[var].quantile(0.75)
            iqr = q3 - q1
            lower_bound = q1 - 1.5 * iqr
            upper_bound = q3 + 1.5 * iqr
            
            # Identifier les outliers
            outlier_col = f'{var_type}_outlier'
            train_df[outlier_col] = ((train_df[var] < lower_bound) | 
                                   (train_df[var] > upper_bound))
            new_target_cols.append(outlier_col)
            
            # Créer version winsorisée
            wins_col = f'{var_type}_winsorized'
            train_df[wins_col] = train_df[var].clip(lower_bound, upper_bound)
            new_target_cols.append(wins_col)
            
            # Statistiques
            target_stats[f'{var_type}_outliers'] = {
                'n_outliers': train_df[outlier_col].sum(),
                'pct_outliers': train_df[outlier_col].mean() * 100,
                'lower_bound': lower_bound,
                'upper_bound': upper_bound
            }
    
    # 3. Création d'indicateurs de risque
    print("\nCréation d'indicateurs de risque:")
    
    # Ratio coût/fréquence
    if freq_var in train_df.columns and cost_var in train_df.columns:
        train_df['cost_freq_ratio'] = train_df[cost_var] / train_df[freq_var].replace(0, np.nan)
        new_target_cols.append('cost_freq_ratio')
    
    # Charge par année
    if charge_var in train_df.columns and 'ANNEE_ASSURANCE' in train_df.columns:
        train_df['charge_per_year'] = train_df[charge_var] / train_df['ANNEE_ASSURANCE']
        new_target_cols.append('charge_per_year')
    
    # 4. Gestion des valeurs manquantes
    print("\nAnalyse des valeurs manquantes:")
    
    for var, var_type in zip([freq_var, cost_var, charge_var], ['freq', 'cost', 'charge']):
        if var in train_df.columns:
            missing_pct = train_df[var].isnull().mean() * 100
            target_stats[f'{var_type}_missing'] = {
                'n_missing': train_df[var].isnull().sum(),
                'pct_missing': missing_pct
            }
            print(f"{var}: {missing_pct:.2f}% de valeurs manquantes")
    
    # 5. Création de bins pour les variables continues
    print("\nCréation de bins:")
    
    for var, var_type in zip([freq_var, cost_var, charge_var], ['freq', 'cost', 'charge']):
        if var in train_df.columns:
            # Créer des bins (10 catégories)
            bins_col = f'{var_type}_bins'
            train_df[bins_col] = pd.qcut(train_df[var], q=10, labels=False, duplicates='drop')
            new_target_cols.append(bins_col)
            
            # Encoder en one-hot si nécessaire
            dummies = pd.get_dummies(train_df[bins_col], prefix=f'{var_type}_bin')
            train_df = pd.concat([train_df, dummies], axis=1)
            new_target_cols.extend(dummies.columns)
    
    # Statistiques finales
    print("\n=== Statistiques des nouvelles variables cibles ===")
    print(f"\nNombre de variables créées: {len(new_target_cols)}")
    
    print("\nStatistiques descriptives des variables numériques:")
    numeric_cols = [col for col in new_target_cols 
                   if train_df[col].dtype in ['int64', 'float64']]
    print(train_df[numeric_cols].describe())
    
    return {
        'new_cols': new_target_cols,
        'train_df': train_df,
        'test_df': test_df,
        'stats': target_stats
    }

# Exécuter la fonction
target_results = handle_target_vars(train_df, test_df)

# Mettre à jour les DataFrames
train_df = target_results['train_df']
test_df = target_results['test_df']

# Afficher les statistiques détaillées
print("\n=== Résumé des statistiques ===")
for category, stats in target_results['stats'].items():
    print(f"\n{category}:")
    for key, value in stats.items():
        print(f"  {key}: {value}")

In [ ]:
def handle_id_vars(train_df, test_df):
    print("=== Traitement des variables d'identification ===")
    
    id_stats = {}
    id_var = ID_VARS['identifiers'][0]  # 'ID'
    properties = ID_VARS['properties']
    
    # 1. Vérification de base des IDs
    print("\nVérification de base des IDs:")
    
    # Vérifier la présence de l'ID
    if id_var not in train_df.columns or id_var not in test_df.columns:
        raise ValueError(f"Variable {id_var} non trouvée dans les données")
    
    # Statistiques initiales
    id_stats['initial'] = {
        'train_rows': len(train_df),
        'test_rows': len(test_df),
        'train_unique_ids': train_df[id_var].nunique(),
        'test_unique_ids': test_df[id_var].nunique()
    }
    
    # 2. Vérification de l'unicité
    print("\nVérification de l'unicité:")
    
    if properties['unique']:
        # Train
        train_duplicates = train_df[train_df[id_var].duplicated(keep=False)]
        n_train_dupes = len(train_duplicates)
        
        # Test
        test_duplicates = test_df[test_df[id_var].duplicated(keep=False)]
        n_test_dupes = len(test_duplicates)
        
        id_stats['duplicates'] = {
            'train_duplicates': n_train_dupes,
            'test_duplicates': n_test_dupes
        }
        
        if n_train_dupes > 0 or n_test_dupes > 0:
            print(f"ATTENTION! Doublons trouvés:")
            print(f"  Train: {n_train_dupes} lignes")
            print(f"  Test: {n_test_dupes} lignes")
            
            # Créer un indicateur de doublon
            train_df['is_duplicate'] = train_df[id_var].duplicated(keep=False)
            test_df['is_duplicate'] = test_df[id_var].duplicated(keep=False)
    
    # 3. Vérification des valeurs nulles
    print("\nVérification des valeurs nulles:")
    
    if properties['not_null']:
        # Train
        train_nulls = train_df[id_var].isnull().sum()
        test_nulls = test_df[id_var].isnull().sum()
        
        id_stats['null_values'] = {
            'train_nulls': train_nulls,
            'test_nulls': test_nulls
        }
        
        if train_nulls > 0 or test_nulls > 0:
            print(f"ATTENTION! Valeurs nulles trouvées:")
            print(f"  Train: {train_nulls} lignes")
            print(f"  Test: {test_nulls} lignes")
    
    # 4. Vérification du type de données
    print("\nVérification du type de données:")
    
    expected_type = properties['type']
    train_type = train_df[id_var].dtype
    test_type = test_df[id_var].dtype
    
    id_stats['data_types'] = {
        'expected': expected_type,
        'train_actual': str(train_type),
        'test_actual': str(test_type)
    }
    
    if expected_type == 'str':
        if not pd.api.types.is_string_dtype(train_type):
            print(f"ATTENTION! Type incorrect dans train: {train_type}")
            train_df[id_var] = train_df[id_var].astype(str)
        
        if not pd.api.types.is_string_dtype(test_type):
            print(f"ATTENTION! Type incorrect dans test: {test_type}")
            test_df[id_var] = test_df[id_var].astype(str)
    
    # 5. Analyse du format des IDs
    print("\nAnalyse du format des IDs:")
    
    # Longueur des IDs
    train_lengths = train_df[id_var].astype(str).str.len()
    test_lengths = test_df[id_var].astype(str).str.len()
    
    id_stats['length_stats'] = {
        'train_min_length': train_lengths.min(),
        'train_max_length': train_lengths.max(),
        'test_min_length': test_lengths.min(),
        'test_max_length': test_lengths.max()
    }
    
    if train_lengths.min() != train_lengths.max():
        print(f"ATTENTION! Longueurs variables dans train: {train_lengths.min()}-{train_lengths.max()}")
    
    if test_lengths.min() != test_lengths.max():
        print(f"ATTENTION! Longueurs variables dans test: {test_lengths.min()}-{test_lengths.max()}")
    
    # 6. Vérification de la cohérence train/test
    print("\nVérification de la cohérence train/test:")
    
    common_ids = set(train_df[id_var]).intersection(set(test_df[id_var]))
    id_stats['train_test_overlap'] = {
        'n_common_ids': len(common_ids),
        'pct_train': len(common_ids) / len(train_df) * 100,
        'pct_test': len(common_ids) / len(test_df) * 100
    }
    
    if common_ids:
        print(f"ATTENTION! {len(common_ids)} IDs communs entre train et test")
        
        # Marquer les IDs communs
        train_df['in_test'] = train_df[id_var].isin(test_df[id_var])
        test_df['in_train'] = test_df[id_var].isin(train_df[id_var])
    
    # 7. Création de hash (si nécessaire)
    if properties.get('create_hash', False):
        print("\nCréation de hash des IDs:")
        
        train_df['id_hash'] = train_df[id_var].apply(lambda x: hash(str(x)))
        test_df['id_hash'] = test_df[id_var].apply(lambda x: hash(str(x)))
    
    # Résumé final
    print("\n=== Résumé des vérifications ===")
    for category, stats in id_stats.items():
        print(f"\n{category}:")
        for key, value in stats.items():
            print(f"  {key}: {value}")
    
    return {
        'train_df': train_df,
        'test_df': test_df,
        'stats': id_stats,
        'has_issues': (n_train_dupes > 0 or n_test_dupes > 0 or 
                      train_nulls > 0 or test_nulls > 0 or 
                      len(common_ids) > 0)
    }

# Exécuter la fonction
id_results = handle_id_vars(train_df, test_df)

# Mettre à jour les DataFrames
train_df = id_results['train_df']
test_df = id_results['test_df']

# Si des problèmes ont été détectés
if id_results['has_issues']:
    print("\nATTENTION! Des problèmes ont été détectés avec les IDs.")
    print("Veuillez vérifier les détails ci-dessus et prendre les mesures appropriées.")

In [ ]:
def clean_and_consolidate_data(train_df, test_df):
    print("=== Nettoyage et consolidation des données ===")
    
    # 1. Traiter chaque groupe de variables
    print("\n1. Traitement des variables par groupe:")
    
    # Weather
    print("\nTraitement des variables météorologiques...")
    weather_results = handle_weather_vars(train_df, test_df)
    train_df = weather_results['train_df']
    test_df = weather_results['test_df']
    
    # Building
    print("\nTraitement des variables bâtiment...")
    building_results = handle_building_vars(train_df, test_df)
    train_df = building_results['train_df']
    test_df = building_results['test_df']
    
    # Geographic
    print("\nTraitement des variables géographiques...")
    geographic_results = handle_geographic_vars(train_df, test_df)
    train_df = geographic_results['train_df']
    test_df = geographic_results['test_df']
    
    # Insurance
    print("\nTraitement des variables d'assurance...")
    insurance_results = handle_insurance_vars(train_df, test_df)
    train_df = insurance_results['train_df']
    test_df = insurance_results['test_df']
    
    # Demographic
    print("\nTraitement des variables démographiques...")
    demographic_results = handle_demographic_vars(train_df, test_df)
    train_df = demographic_results['train_df']
    test_df = demographic_results['test_df']
    
    # Activity
    print("\nTraitement des variables d'activité...")
    activity_results = handle_activity_vars(train_df, test_df)
    train_df = activity_results['train_df']
    test_df = activity_results['test_df']
    
    # Target (seulement pour train_df)
    print("\nTraitement des variables cibles...")
    target_results = handle_target_vars(train_df, test_df)
    train_df = target_results['train_df']
    test_df = target_results['test_df']
    
    # ID
    print("\nTraitement des variables d'identification...")
    id_results = handle_id_vars(train_df, test_df)
    train_df = id_results['train_df']
    test_df = id_results['test_df']
    
    # 2. Consolider les colonnes à supprimer
    cols_to_drop = []
    cols_to_drop.extend(weather_results.get('to_drop', []))
    cols_to_drop.extend(building_results.get('to_drop', []))
    cols_to_drop.extend(geographic_results.get('to_drop', []))
    cols_to_drop.extend(insurance_results.get('to_drop', []))
    cols_to_drop.extend(demographic_results.get('to_drop', []))
    cols_to_drop.extend(activity_results.get('to_drop', []))
    
    # 3. Consolider les nouvelles colonnes
    new_cols = []
    new_cols.extend(weather_results.get('new_cols', []))
    new_cols.extend(building_results.get('new_cols', []))
    new_cols.extend(geographic_results.get('new_cols', []))
    new_cols.extend(insurance_results.get('new_cols', []))
    new_cols.extend(demographic_results.get('new_cols', []))
    new_cols.extend(activity_results.get('new_cols', []))
    new_cols.extend(target_results.get('new_cols', []))
    
    # 4. Vérifier la qualité des données consolidées
    print("\n=== Vérification finale des données ===")
    
    # Statistiques sur les nouvelles variables
    print("\nStatistiques des nouvelles variables numériques:")
    numeric_cols = [col for col in new_cols if train_df[col].dtype in ['int64', 'float64']]
    print(train_df[numeric_cols].describe())
    
    # Valeurs manquantes
    missing_pct = (train_df[new_cols].isnull().mean() * 100).round(2)
    print("\nPourcentage de valeurs manquantes dans les nouvelles variables:")
    print(missing_pct[missing_pct > 0])
    
    # Dimensions finales
    print("\nDimensions finales:")
    print(f"Train: {train_df.shape}")
    print(f"Test: {test_df.shape}")
    
    # 5. Supprimer les anciennes colonnes
    train_df_clean = train_df.drop(columns=cols_to_drop)
    test_df_clean = test_df.drop(columns=cols_to_drop)
    
    return {
        'train_df': train_df_clean,
        'test_df': test_df_clean,
        'new_cols': new_cols,
        'dropped_cols': cols_to_drop,
        'stats': {
            'n_new_cols': len(new_cols),
            'n_dropped_cols': len(cols_to_drop),
            'train_shape': train_df_clean.shape,
            'test_shape': test_df_clean.shape,
            'missing_stats': missing_pct[missing_pct > 0].to_dict()
        }
    }

# Exécuter le nettoyage
results = clean_and_consolidate_data(train_df, test_df)

# Mettre à jour les DataFrames
train_df = results['train_df']
test_df = results['test_df']

# Afficher le résumé
print("\n=== Résumé des modifications ===")
print(f"Nombre de nouvelles colonnes créées: {results['stats']['n_new_cols']}")
print(f"Nombre de colonnes supprimées: {results['stats']['n_dropped_cols']}")
print("\nDimensions finales:")
print(f"Train: {results['stats']['train_shape']}")
print(f"Test: {results['stats']['test_shape']}")

def process_weather_data(X_train, X_test, missing_summary, weather_cols):
    """
    Pipeline complet pour le traitement des variables météo:
    1. Nettoyage et imputation des variables individuelles
    2. Création de variables agrégées 
    """
    # Étape 1: Nettoyage initial et imputation
    X_train_clean, X_test_clean, missing_indicators = clean_weather_vars(
        X_train, X_test, missing_summary, weather_cols
    )
    
    # Étape 2: Création de variables agrégées
    weather_results = aggregate_weather_vars(X_train_clean, X_test_clean)
    
    # Collecter toutes les nouvelles variables créées
    new_cols = weather_results['new_cols']
    cols_to_drop = weather_results['to_drop']
    
    # Récupérer les DataFrames finaux
    X_train_final = weather_results['train_df'] if 'train_df' in weather_results else X_train_clean
    X_test_final = weather_results['test_df'] if 'test_df' in weather_results else X_test_clean
    
    # Ajouter les indicateurs de valeurs manquantes
    for col, indicator in missing_indicators.items():
        X_train_final[col] = indicator
        X_test_final[col] = indicator  # Appliquer aussi au jeu de test
    
    return {
        'train_df': X_train_final,
        'test_df': X_test_final,
        'new_cols': new_cols,
        'to_drop': cols_to_drop,
        'missing_indicators': missing_indicators
    }

def process_building_data(X_train, X_test, missing_summary, building_cols):
    """
    Pipeline complet pour le traitement des variables bâtiment:
    1. Nettoyage et imputation des variables individuelles
    2. Création de variables agrégées et dérivées
    """
    # Étape 1: Nettoyage initial et imputation
    X_train_clean, X_test_clean, missing_indicators = clean_building_vars(
        X_train, X_test, missing_summary, building_cols
    )
    
    # Étape 2: Création de variables agrégées
    building_results = aggregate_building_vars(X_train_clean, X_test_clean)
    
    # Collecter toutes les nouvelles variables créées
    new_cols = building_results['new_cols']
    cols_to_drop = building_results['to_drop']
    
    # Récupérer les DataFrames finaux
    X_train_final = building_results['train_df']
    X_test_final = building_results['test_df']
    
    # Ajouter les indicateurs de valeurs manquantes
    for col, indicator in missing_indicators.items():
        X_train_final[col] = indicator
        X_test_final[col] = indicator  # Appliquer aussi au jeu de test
    
    return {
        'train_df': X_train_final,
        'test_df': X_test_final,
        'new_cols': new_cols,
        'to_drop': cols_to_drop,
        'missing_indicators': missing_indicators
    }

def clean_building_vars(X_train, X_test, missing_summary, building_cols):
    """
    Traite les variables bâtiment avec vérification détaillée des types
    et imputation des valeurs manquantes
    """
    missing_indicators = {}
    print("\nNettoyage des variables bâtiment:")
    
    for col in building_cols:
        if col in X_train.columns:
            missing_pct = missing_summary.loc[col, 'Missing Percentage']
            print(f"\nColonne {col}:")
            print(f"- Pourcentage de valeurs manquantes: {missing_pct:.2f}%")
            print(f"- Type initial: {X_train[col].dtype}")
            
            # Créer indicateur de valeurs manquantes
            if missing_pct > 0:
                missing_indicators[f'{col}_is_missing'] = X_train[col].isnull().astype(int)
            
            if missing_pct <= 70:  # On traite uniquement si moins de 70% manquant
                if X_train[col].dtype in ['int64', 'float64']:
                    # Pour les variables numériques
                    median_val = X_train[col].median()
                    X_train[col].fillna(median_val, inplace=True)
                    X_test[col].fillna(median_val, inplace=True)
                    print(f"- Variable numérique, imputation avec la médiane: {median_val}")
                    print(f"- Plage de valeurs: [{X_train[col].min()}, {X_train[col].max()}]")
                else:
                    # Pour les variables catégorielles
                    # Filtrer les valeurs non-nulles avant de les afficher
                    unique_vals = X_train[col].dropna().unique()
                    print("- Variable catégorielle, valeurs uniques:")
                    print(unique_vals[:5])
                    
                    # Si peu de valeurs uniques, on pourrait considérer une conversion en numérique
                    if len(unique_vals) < 10:
                        print(f"- Nombre de catégories: {len(unique_vals)}")
                        print("- Distribution des valeurs:")
                        print(X_train[col].value_counts().head())
                    
                    X_train[col].fillna('UNKNOWN', inplace=True)
                    X_test[col].fillna('UNKNOWN', inplace=True)
                    print(f"- Imputation avec 'UNKNOWN'")
            else:
                print("- Colonne ignorée (>70% manquant)")
            
            # Vérification après traitement
            missing_after = X_train[col].isnull().sum()
            if missing_after > 0:
                print(f"⚠️ ATTENTION: {missing_after} valeurs manquantes restantes!")
    
    return X_train, X_test, missing_indicators

def aggregate_building_vars(train_df, test_df):
    """
    Crée des variables agrégées et des indicateurs composites à partir
    des variables bâtiment nettoyées
    """
    print("\n=== Création de variables bâtiment agrégées ===")
    
    new_building_cols = []
    
    # 1. Consolider les hauteurs
    height_cols = BUILDING_VARS['height']
    if height_cols:
        print("\nConsolidation des hauteurs:")
        height_aggs = {
            'height_mean': 'mean',
            'height_max': 'max',
            'height_min': 'min'
        }
        
        for col_name, agg_func in height_aggs.items():
            train_df[col_name] = train_df[height_cols].agg(agg_func, axis=1)
            test_df[col_name] = test_df[height_cols].agg(agg_func, axis=1)
            new_building_cols.append(col_name)
        
        print(f"Créé {len(height_aggs)} variables de hauteur agrégées")
    
    # 2. Consolider les surfaces
    surface_cols = BUILDING_VARS['surface']
    if surface_cols:
        print("\nConsolidation des surfaces:")
        surface_aggs = {
            'surface_total': 'sum',
            'surface_mean': 'mean',
            'surface_max': 'max',
            'n_surfaces': lambda x: x.notna().sum()  # Nombre de surfaces renseignées
        }
        
        for col_name, agg_func in surface_aggs.items():
            train_df[col_name] = train_df[surface_cols].agg(agg_func, axis=1)
            test_df[col_name] = test_df[surface_cols].agg(agg_func, axis=1)
            new_building_cols.append(col_name)
        
        print(f"Créé {len(surface_aggs)} variables de surface agrégées")
    
    # 3. Consolider le nombre de bâtiments
    building_cols = BUILDING_VARS['buildings']
    if building_cols:
        print("\nConsolidation du nombre de bâtiments:")
        building_aggs = {
            'buildings_total': 'sum',
            'buildings_count': lambda x: x.notna().sum(),  # Nombre de types de bâtiments
            'buildings_types': lambda x: (x > 0).sum()     # Nombre de types avec bâtiments
        }
        
        for col_name, agg_func in building_aggs.items():
            train_df[col_name] = train_df[building_cols].agg(agg_func, axis=1)
            test_df[col_name] = test_df[building_cols].agg(agg_func, axis=1)
            new_building_cols.append(col_name)
        
        print(f"Créé {len(building_aggs)} variables de comptage de bâtiments")
    
    # 4. Créer des ratios et indicateurs composites
    print("\nCréation d'indicateurs composites:")
    
    # Ratio surface/bâtiment
    if 'surface_total' in new_building_cols and 'buildings_total' in new_building_cols:
        train_df['surface_per_building'] = (train_df['surface_total'] / 
                                          train_df['buildings_total'].replace(0, np.nan))
        test_df['surface_per_building'] = (test_df['surface_total'] / 
                                         test_df['buildings_total'].replace(0, np.nan))
        new_building_cols.append('surface_per_building')
    
    # Ratio hauteur/surface
    if 'height_mean' in new_building_cols and 'surface_total' in new_building_cols:
        train_df['height_surface_ratio'] = (train_df['height_mean'] / 
                                          train_df['surface_total'].replace(0, np.nan))
        test_df['height_surface_ratio'] = (test_df['height_mean'] / 
                                         test_df['surface_total'].replace(0, np.nan))
        new_building_cols.append('height_surface_ratio')
    
    # 5. Encoder les caractéristiques catégorielles
    char_cols = BUILDING_VARS['characteristics']
    if char_cols:
        print("\nEncodage des caractéristiques:")
        for col in char_cols:
            dummies = pd.get_dummies(train_df[col], prefix=f'char_{col}')
            test_dummies = pd.get_dummies(test_df[col], prefix=f'char_{col}')
            
            # Assurer les mêmes colonnes dans train et test
            for dummy_col in dummies.columns:
                if dummy_col not in test_dummies.columns:
                    test_dummies[dummy_col] = 0
            
            train_df = pd.concat([train_df, dummies], axis=1)
            test_df = pd.concat([test_df, test_dummies], axis=1)
            new_building_cols.extend(dummies.columns)
        
        print(f"Créé {len(dummies.columns)} variables dummy pour les caractéristiques")
    
    # Identifier les colonnes à supprimer
    building_cols_to_drop = sum([
        BUILDING_VARS['height'],
        BUILDING_VARS['surface'],
        BUILDING_VARS['buildings'],
        BUILDING_VARS['characteristics']
    ], [])
    
    # Statistiques des nouvelles variables
    print("\n=== Statistiques des nouvelles variables bâtiment ===")
    print(f"\nNombre de variables créées: {len(new_building_cols)}")
    print(f"Nombre de variables à supprimer: {len(building_cols_to_drop)}")
    
    print("\nStatistiques descriptives des variables numériques:")
    numeric_cols = [col for col in new_building_cols 
                   if train_df[col].dtype in ['int64', 'float64']]
    print(train_df[numeric_cols].describe())
    
    return {
        'to_drop': building_cols_to_drop,
        'new_cols': new_building_cols,
        'train_df': train_df,
        'test_df': test_df
    }

# Utilisation de la fonction fusionnée
building_data = process_building_data(
    X_train, X_test, missing_summary, groups['Building']
)

# Mise à jour des DataFrames
X_train = building_data['train_df']
X_test = building_data['test_df']

In [ ]:
def handle_activity_vars(train_df, test_df):
    print("=== Traitement des variables d'activité ===")
    
    new_activity_cols = []
    
    # 1. Traitement des équipements
    if 'equipment' in ACTIVITY_VARS:
        print("\nTraitement des équipements:")
        equip_cols = ACTIVITY_VARS['equipment']
        
        # Nombre total d'équipements
        train_df['n_equipements_total'] = (train_df[equip_cols] > 0).sum(axis=1)
        test_df['n_equipements_total'] = (test_df[equip_cols] > 0).sum(axis=1)
        new_activity_cols.append('n_equipements_total')
        
        # Score de complexité des équipements (pondéré par le numéro d'équipement)
        equip_weights = {col: int(col.replace('EQUIPEMENT', '')) for col in equip_cols}
        train_df['score_complexite_equip'] = sum(train_df[col] * weight 
                                               for col, weight in equip_weights.items())
        test_df['score_complexite_equip'] = sum(test_df[col] * weight 
                                              for col, weight in equip_weights.items())
        new_activity_cols.append('score_complexite_equip')
        
        # Encodage one-hot des équipements
        for col in equip_cols:
            dummies = pd.get_dummies(train_df[col], prefix=f'equip_{col}')
            test_dummies = pd.get_dummies(test_df[col], prefix=f'equip_{col}')
            
            # Assurer les mêmes colonnes
            for dummy_col in dummies.columns:
                if dummy_col not in test_dummies.columns:
                    test_dummies[dummy_col] = 0
            
            train_df = pd.concat([train_df, dummies], axis=1)
            test_df = pd.concat([test_df, test_dummies], axis=1)
            new_activity_cols.extend(dummies.columns)
    
    # 2. Traitement de l'activité et vocation
    if 'activity' in ACTIVITY_VARS:
        print("\nTraitement de l'activité et vocation:")
        activity_cols = ACTIVITY_VARS['activity']
        
        # Encodage des variables catégorielles
        for col in activity_cols:
            # Encodage avec gestion de la fréquence
            encoding_map = (train_df[col].value_counts() / len(train_df)).to_dict()
            
            # Appliquer l'encodage
            train_df[f'{col}_freq'] = train_df[col].map(encoding_map)
            test_df[f'{col}_freq'] = test_df[col].map(encoding_map)
            new_activity_cols.append(f'{col}_freq')
            
            # Encodage one-hot classique
            dummies = pd.get_dummies(train_df[col], prefix=col)
            test_dummies = pd.get_dummies(test_df[col], prefix=col)
            
            # Assurer les mêmes colonnes
            for dummy_col in dummies.columns:
                if dummy_col not in test_dummies.columns:
                    test_dummies[dummy_col] = 0
            
            train_df = pd.concat([train_df, dummies], axis=1)
            test_df = pd.concat([test_df, test_dummies], axis=1)
            new_activity_cols.extend(dummies.columns)
        
        # Créer des combinaisons d'activité-vocation si disponibles
        if 'ACTIVIT2' in activity_cols and 'VOCATION' in activity_cols:
            train_df['activ_voc'] = train_df['ACTIVIT2'] + '_' + train_df['VOCATION']
            test_df['activ_voc'] = test_df['ACTIVIT2'] + '_' + test_df['VOCATION']
            
            # Encoder cette nouvelle combinaison
            dummies = pd.get_dummies(train_df['activ_voc'], prefix='activ_voc')
            test_dummies = pd.get_dummies(test_df['activ_voc'], prefix='activ_voc')
            
            # Assurer les mêmes colonnes
            for dummy_col in dummies.columns:
                if dummy_col not in test_dummies.columns:
                    test_dummies[dummy_col] = 0
            
            train_df = pd.concat([train_df, dummies], axis=1)
            test_df = pd.concat([test_df, test_dummies], axis=1)
            new_activity_cols.extend(dummies.columns)
    
    # 3. Traitement de la taille et catégorie
    if 'size_category' in ACTIVITY_VARS:
        print("\nTraitement de la taille et catégorie:")
        size_cols = [col for col in ACTIVITY_VARS['size_category'] if col.startswith('TAILLE')]
        
        # Score de taille global
        if size_cols:
            train_df['score_taille'] = train_df[size_cols].mean(axis=1)
            test_df['score_taille'] = test_df[size_cols].mean(axis=1)
            new_activity_cols.append('score_taille')
            
            # Variabilité de la taille
            train_df['var_taille'] = train_df[size_cols].std(axis=1)
            test_df['var_taille'] = test_df[size_cols].std(axis=1)
            new_activity_cols.append('var_taille')
        
        # Traitement de COEFASS
        if 'COEFASS' in ACTIVITY_VARS['size_category']:
            # Encodage fréquentiel
            coef_map = (train_df['COEFASS'].value_counts() / len(train_df)).to_dict()
            train_df['coefass_freq'] = train_df['COEFASS'].map(coef_map)
            test_df['coefass_freq'] = test_df['COEFASS'].map(coef_map)
            new_activity_cols.append('coefass_freq')
            
            # Encodage one-hot
            dummies = pd.get_dummies(train_df['COEFASS'], prefix='coefass')
            test_dummies = pd.get_dummies(test_df['COEFASS'], prefix='coefass')
            
            # Assurer les mêmes colonnes
            for dummy_col in dummies.columns:
                if dummy_col not in test_dummies.columns:
                    test_dummies[dummy_col] = 0
            
            train_df = pd.concat([train_df, dummies], axis=1)
            test_df = pd.concat([test_df, test_dummies], axis=1)
            new_activity_cols.extend(dummies.columns)
    
    # 4. Traitement des services d'urgence
    if 'emergency' in ACTIVITY_VARS:
        print("\nTraitement des services d'urgence:")
        emergency_cols = ACTIVITY_VARS['emergency']
        # Garder ces variables telles quelles
        new_activity_cols.extend(emergency_cols)
    
    # Identifier les colonnes à supprimer
    activity_cols_to_drop = sum([
        ACTIVITY_VARS['equipment'],
        ACTIVITY_VARS['activity'],
        ACTIVITY_VARS['size_category']
    ], [])
    
    # Statistiques des nouvelles variables
    print("\n=== Statistiques des nouvelles variables d'activité ===")
    print(f"\nNombre de variables créées: {len(new_activity_cols)}")
    print(f"Nombre de variables à supprimer: {len(activity_cols_to_drop)}")
    
    print("\nStatistiques descriptives des variables numériques:")
    numeric_cols = [col for col in new_activity_cols 
                   if train_df[col].dtype in ['int64', 'float64']]
    print(train_df[numeric_cols].describe())
    
    return {
        'to_drop': activity_cols_to_drop,
        'new_cols': new_activity_cols,
        'train_df': train_df,
        'test_df': test_df
    }

# Exécuter la fonction
activity_results = handle_activity_vars(train_df, test_df)

# Mettre à jour les DataFrames
train_df = activity_results['train_df']
test_df = activity_results['test_df']

In [ ]:
def verify_variable_groups():
    print("=== Vérification des groupes de variables ===")
    
    # Fonction récursive pour extraire toutes les variables d'une structure imbriquée
    def extract_variables(structure):
        variables = set()
        if isinstance(structure, dict):
            for value in structure.values():
                if isinstance(value, (list, set)):
                    variables.update(value)
                elif isinstance(value, dict):
                    variables.update(extract_variables(value))
        elif isinstance(structure, (list, set)):
            variables.update(structure)
        return variables
    
    # Rassembler toutes les variables des groupes
    group_stats = {}
    all_grouped_vars = set()
    
    # Traiter chaque groupe principal
    groups_dict = {
        'WEATHER': WEATHER_VARS,
        'BUILDING': BUILDING_VARS,
        'INSURANCE': INSURANCE_VARS,
        'GEOGRAPHIC': GEOGRAPHIC_VARS,
        'DEMOGRAPHIC': DEMOGRAPHIC_VARS,
        'ACTIVITY': ACTIVITY_VARS,
        'TARGET': TARGET_VARS['primary'],  # Cas spécial pour TARGET_VARS
        'ID': ID_VARS['identifiers']       # Cas spécial pour ID_VARS
    }
    
    # Analyser chaque groupe
    for group_name, group_structure in groups_dict.items():
        group_vars = extract_variables(group_structure)
        group_stats[group_name] = {
            'count': len(group_vars),
            'variables': sorted(group_vars)
        }
        all_grouped_vars.update(group_vars)
    
    # Comparer avec les variables du dataset
    all_df_vars = set(train_df.columns)
    ungrouped_vars = all_df_vars - all_grouped_vars
    overlapping_vars = set()
    
    # Vérifier les chevauchements entre groupes
    for group1 in groups_dict.keys():
        for group2 in groups_dict.keys():
            if group1 < group2:  # Éviter les comparaisons doubles
                vars1 = set(group_stats[group1]['variables'])
                vars2 = set(group_stats[group2]['variables'])
                overlap = vars1.intersection(vars2)
                if overlap:
                    overlapping_vars.update(overlap)
                    print(f"\nChevauchement entre {group1} et {group2}:")
                    print(sorted(overlap))
    
    # Afficher les statistiques
    print("\n=== Statistiques par groupe ===")
    for group_name, stats in group_stats.items():
        print(f"\n{group_name}:")
        print(f"  Nombre de variables: {stats['count']}")
        print(f"  Exemples: {stats['variables'][:3]}...")
    
    print("\n=== Résumé global ===")
    print(f"Variables dans le dataset: {len(all_df_vars)}")
    print(f"Variables groupées: {len(all_grouped_vars)}")
    print(f"Variables non groupées: {len(ungrouped_vars)}")
    print(f"Variables en chevauchement: {len(overlapping_vars)}")
    
    if ungrouped_vars:
        print("\n=== Variables non groupées ===")
        print(sorted(ungrouped_vars))
    
    if overlapping_vars:
        print("\n=== Variables en chevauchement ===")
        print(sorted(overlapping_vars))
    
    return {
        'group_stats': group_stats,
        'summary': {
            'total_vars': len(all_df_vars),
            'grouped_vars': len(all_grouped_vars),
            'ungrouped_vars': sorted(ungrouped_vars),
            'overlapping_vars': sorted(overlapping_vars)
        }
    }

# Exécuter la vérification
verification_results = verify_variable_groups()

# Afficher des recommandations si nécessaire
if verification_results['summary']['ungrouped_vars']:
    print("\n=== Recommandations ===")
    print("Variables à grouper:")
    for var in verification_results['summary']['ungrouped_vars']:
        # Suggérer un groupe basé sur le nom de la variable
        if any(x in var.upper() for x in ['TEMP', 'WIND', 'RAIN']):
            print(f"  {var} -> WEATHER_VARS")
        elif any(x in var.upper() for x in ['BUILD', 'SURFACE', 'HEIGHT']):
            print(f"  {var} -> BUILDING_VARS")
        elif any(x in var.upper() for x in ['RISK', 'INSURANCE', 'CAPITAL']):
            print(f"  {var} -> INSURANCE_VARS")
        else:
            print(f"  {var} -> Groupe à déterminer")